https://drive.google.com/drive/u/0/folders/1X_ag69ToGRCF32K2a099pgvvNMg0j8nq

In [44]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

date_format="%d/%m/%Y %H:%M"
date_parse = lambda date: dt.datetime.strptime(date, date_format)
# dat = pd.read_csv("/home/ciaran/knn_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/lasso_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/lgbm_Q_DAM_1-12.csv")
dat = pd.read_csv("/home/ciaran/rf_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/SH_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/MH_Q_DAM_1-12.csv")

dat1=pd.DataFrame(dat)

column_names = ['EURPrices+{}_Forecast_10'.format(i) for i in range(0, 24)]
Q_10=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_30'.format(i) for i in range(0, 24)]
Q_30=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_50'.format(i) for i in range(0, 24)]
Q_50=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_70'.format(i) for i in range(0, 24)]
Q_70=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_90'.format(i) for i in range(0, 24)]
Q_90=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}'.format(i) for i in range(0, 24)]
Y_r=dat1[column_names].dropna().stack().reset_index()
Y_r=Y_r.iloc[:,:]

Y_r["Price"]=Y_r.iloc[:,2:3]
Q_10["Price"]=Q_10.iloc[:,2:3]
Q_30["Price"]=Q_30.iloc[:,2:3]
Q_50["Price"]=Q_50.iloc[:,2:3]
Q_70["Price"]=Q_70.iloc[:,2:3]
Q_90["Price"]=Q_90.iloc[:,2:3]

In [45]:
def run_electricity_strategy(df, Q_A_Preds, Q_B_Preds,  eff_1, eff_2, capacity,charge_level, ramp_rate, min_charge_level):
    prices = []
    charge_level = charge_level    
    level_0_values = df['level_0'].unique()

    for level_0 in level_0_values:
        current_df = df[df['level_0'] == level_0]
        current_Q_A = Q_A_Preds[Q_A_Preds['level_0'] == level_0]
        current_Q_B = Q_B_Preds[Q_B_Preds['level_0'] == level_0]
        
        max_price_index = current_Q_A['Price'].idxmax()
        min_price_index = current_Q_B['Price'].idxmin()

        smaller_index = min(min_price_index, max_price_index)
        larger_index = max(min_price_index, max_price_index)
        
        remaining_prices_0 = current_df[(current_df.index > smaller_index) & (current_df.index > larger_index)]
        remaining_prices_0_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index) & (Q_A_Preds.index > larger_index)]
        remaining_prices_0_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index) & (Q_B_Preds.index > larger_index)]

        remaining_prices_1 = current_df[(current_df.index > smaller_index) & (current_df.index < larger_index)]
        remaining_prices_1_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index) & (Q_A_Preds.index < larger_index)]
        remaining_prices_1_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index) & (Q_B_Preds.index < larger_index)]

        remaining_prices_2 = current_df[(current_df.index < smaller_index) & (current_df.index < larger_index)]
        remaining_prices_2_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index < smaller_index) & (Q_A_Preds.index < larger_index)]
        remaining_prices_2_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index < smaller_index) & (Q_B_Preds.index < larger_index)]

        if current_Q_B.loc[min_price_index, 'Price'] < current_Q_A.loc[max_price_index, 'Price']:
            if min_price_index < max_price_index:
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
            
                profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index, 'Price'] * bottleneck_1)/eff_2)
            
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'], max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))
            
            elif min_price_index > max_price_index:
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
            
                profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index, 'Price'] * bottleneck_1)/eff_2)
            
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'],  max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))
            
            else:
                continue
                                    
      
    
        if len(remaining_prices_0) > 1:
            max_price_index_1 = remaining_prices_0_A['Price'].idxmax()
            min_price_index_1 = remaining_prices_0_B['Price'].idxmin()
            charge_level=charge_level
            if current_Q_B.loc[min_price_index_1, 'Price'] < current_Q_A.loc[max_price_index_1, 'Price']:
                  if min_price_index_1 < max_price_index_1:
                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                    charge_level += bottleneck_1
                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                    charge_level -= bottleneck_2
            
                    profit = (current_df.loc[max_price_index_1, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)/eff_2)
            
                    prices.append((min_price_index_1, current_df.loc[min_price_index_1, 'Price'], max_price_index_1, current_df.loc[max_price_index_1, 'Price'], profit, charge_level))
            
                  elif min_price_index_1 > max_price_index_1:
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
            
                        profit = (current_df.loc[max_price_index_1, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)/eff_2)
            
                        prices.append((min_price_index_1, current_df.loc[min_price_index_1, 'Price'], max_price_index_1, current_df.loc[max_price_index_1, 'Price'], profit, charge_level))
            
                  else:
                        continue  

                        
            smaller_index_1 = min(min_price_index_1, max_price_index_1)
            larger_index_1 = max(min_price_index_1, max_price_index_1)
            remaining_prices_01 = current_df[(current_df.index > smaller_index_1) & (current_df.index < larger_index_1)]
            remaining_prices_01_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_1) & (Q_A_Preds.index < larger_index_1)]
            remaining_prices_01_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_1) & (Q_B_Preds.index < larger_index_1)]

            if len(remaining_prices_01) > 1:
                max_price_index_4 = remaining_prices_01_A['Price'].idxmax()
                min_price_index_4 = remaining_prices_01_B['Price'].idxmin()
                if current_Q_B.loc[min_price_index_4, 'Price'] < current_Q_A.loc[max_price_index_4, 'Price']:
                    if min_price_index_4 < max_price_index_4:
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                    
                        profit = (current_df.loc[max_price_index_4, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_4, 'Price'] * bottleneck_1)/eff_2)
                    
                        prices.append((min_price_index_4, current_df.loc[min_price_index_4, 'Price'], max_price_index_1, current_df.loc[max_price_index_4, 'Price'], profit, charge_level))
                    
                    elif min_price_index_4 > max_price_index_4:
                          bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                          charge_level -= bottleneck_2
                          bottleneck_1 = min(capacity - charge_level, ramp_rate)
                          charge_level += bottleneck_1
                    
                          profit = (current_df.loc[max_price_index_4, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)/eff_2)
                    
                          prices.append((min_price_index_4, current_df.loc[min_price_index_4, 'Price'], max_price_index_4, current_df.loc[max_price_index_4, 'Price'], profit, charge_level))
                                
                    else:
                         continue

                  
        if len(remaining_prices_1) > 1:
                max_price_index_2 = remaining_prices_1_A['Price'].idxmax()
                min_price_index_2 = remaining_prices_1_B['Price'].idxmin()
                if current_Q_B.loc[min_price_index_2, 'Price'] < current_Q_A.loc[max_price_index_2, 'Price']:
                      if min_price_index_2 < max_price_index_2:
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
            
                        profit = (current_df.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_2, 'Price'] * bottleneck_1)/eff_2)
            
                        prices.append((min_price_index_2, current_df.loc[min_price_index_2, 'Price'], max_price_index_2, current_df.loc[max_price_index_2, 'Price'], profit, charge_level))
            
                      elif min_price_index_2 > max_price_index_2:
                         bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                         charge_level -= bottleneck_2
                         bottleneck_1 = min(capacity - charge_level, ramp_rate)
                         charge_level += bottleneck_1
            
                         profit = (current_df.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_2, 'Price'] * bottleneck_1)/eff_2)
            
                         prices.append((min_price_index_2, current_df.loc[min_price_index_2, 'Price'], max_price_index_2, current_df.loc[max_price_index_2, 'Price'], profit, charge_level))
            
                      else:
                            continue



                                            
                     

    return pd.DataFrame(prices, columns=['minPriceIndex', 'minPrice','maxPriceIndex', 'maxPrice', 'profit', 'charge Level'])



r_dam_50_50=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_50, Q_B_Preds=Q_50, eff_1=1, eff_2=1, capacity=5,charge_level=3, ramp_rate=1, min_charge_level=0)
r_dam_10_90=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_10, Q_B_Preds=Q_90, eff_1=1, eff_2=1, capacity=5,charge_level=3, ramp_rate=1, min_charge_level=0)
r_dam_30_70=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_30, Q_B_Preds=Q_70, eff_1=1, eff_2=1, capacity=5,charge_level=3, ramp_rate=1, min_charge_level=0)

In [46]:
print(sum(r_dam_50_50.iloc[:,4:5].values))
r_dam_50_50

[61354.26]


,minPriceIndex,minPrice,maxPriceIndex,maxPrice,profit,charge Level
0,5,12.26,17,31.66,19.40,3
1,23,13.98,18,28.66,14.68,3
2,22,14.26,19,23.22,9.24,3
3,14,15.16,10,25.67,10.51,3
4,28,12.89,42,36.00,23.11,3
...,...,...,...,...,...,...
1316,8716,77.62,8718,88.28,10.66,3
1317,8740,81.63,8744,121.10,39.47,3
1318,8759,83.68,8746,135.88,52.20,3
1319,8758,87.55,8747,135.56,51.88,3


In [47]:
print(sum(r_dam_10_90.iloc[:,4:5].values))
r_dam_10_90

[50694.58]


,minPriceIndex,minPrice,maxPriceIndex,maxPrice,profit,charge Level
0,25,12.64,42,36.00,23.36,3
1,26,12.88,41,39.82,26.94,3
2,50,16.23,66,37.30,21.07,3
3,52,21.21,65,37.16,15.95,3
4,85,15.00,91,33.17,18.17,3
...,...,...,...,...,...,...
833,8710,87.92,8698,204.99,119.99,3
834,8692,87.65,8695,258.87,171.22,3
835,8715,77.62,8731,117.03,39.41,3
836,8737,89.67,8744,121.10,31.43,3


In [48]:
print(sum(r_dam_30_70.iloc[:,4:5].values))
r_dam_30_70

[61438.8]


,minPriceIndex,minPrice,maxPriceIndex,maxPrice,profit,charge Level
0,4,11.58,18,28.66,17.08,3
1,23,13.98,19,23.22,9.24,3
2,22,14.26,20,21.00,7.02,3
3,14,15.16,17,31.66,16.50,3
4,25,12.64,42,36.00,23.36,3
...,...,...,...,...,...,...
1251,8716,77.62,8730,123.22,45.60,3
1252,8740,81.63,8744,121.10,39.47,3
1253,8759,83.68,8745,148.90,65.22,3
1254,8750,83.87,8746,135.88,52.20,3


In [49]:
date_format="%m/%d/%Y %H:%M"
date_parse = lambda date: dt.datetime.strptime(date, date_format)
dat1 = pd.read_csv("/home/ciaran/rf_Q_1-12.csv")
dat1=pd.DataFrame(dat1)
column_names = ['lag_{}y_Forecast_10'.format(i) for i in range(2, 18)]
Q_10=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_30'.format(i) for i in range(2, 18)]
Q_30=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_50'.format(i) for i in range(2, 18)]
Q_50=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_70'.format(i) for i in range(2, 18)]
Q_70=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_90'.format(i) for i in range(2, 18)]
Q_90=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y'.format(i) for i in range(2, 18)]
Y_r=dat1[column_names].dropna().stack().reset_index()
# YY_test= Y_r.iloc[:, 2:]
Y_r["Price"]=Y_r.iloc[:,2:3]
Q_10["Price"]=Q_10.iloc[:,2:3]
Q_30["Price"]=Q_30.iloc[:,2:3]
Q_50["Price"]=Q_50.iloc[:,2:3]
Q_70["Price"]=Q_70.iloc[:,2:3]
Q_90["Price"]=Q_90.iloc[:,2:3]

In [50]:
def run_electricity_strategy(df, Q_A_Preds, Q_B_Preds,  eff_1, eff_2, capacity,charge_level, ramp_rate, min_charge_level):
    prices = []
    charge_level = charge_level    
    level_0_values = df['level_0'].unique()

    for level_0 in level_0_values:
        current_df = df[df['level_0'] == level_0]
        current_Q_A = Q_A_Preds[Q_A_Preds['level_0'] == level_0]
        current_Q_B = Q_B_Preds[Q_B_Preds['level_0'] == level_0]
        
        max_price_index = current_Q_A['Price'].idxmax()
        min_price_index = current_Q_B['Price'].idxmin()

        smaller_index = min(min_price_index, max_price_index)
        larger_index = max(min_price_index, max_price_index)
        
        remaining_prices_0 = current_df[(current_df.index > smaller_index) & (current_df.index > larger_index)]
        remaining_prices_0_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index) & (Q_A_Preds.index > larger_index)]
        remaining_prices_0_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index) & (Q_B_Preds.index > larger_index)]

        remaining_prices_1 = current_df[(current_df.index > smaller_index) & (current_df.index < larger_index)]
        remaining_prices_1_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index) & (Q_A_Preds.index < larger_index)]
        remaining_prices_1_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index) & (Q_B_Preds.index < larger_index)]

        remaining_prices_2 = current_df[(current_df.index < smaller_index) & (current_df.index < larger_index)]
        remaining_prices_2_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index < smaller_index) & (Q_A_Preds.index < larger_index)]
        remaining_prices_2_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index < smaller_index) & (Q_B_Preds.index < larger_index)]

        if current_Q_B.loc[min_price_index, 'Price'] < current_Q_A.loc[max_price_index, 'Price']:
            if min_price_index < max_price_index:
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
            
                profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index, 'Price'] * bottleneck_1)/eff_2)
            
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'], max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))
            
            elif min_price_index > max_price_index:
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
            
                profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index, 'Price'] * bottleneck_1)/eff_2)
            
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'],  max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))
            
            else:
                continue
                                    
      
    
        if len(remaining_prices_0) > 1:
            max_price_index_1 = remaining_prices_0_A['Price'].idxmax()
            min_price_index_1 = remaining_prices_0_B['Price'].idxmin()
            charge_level=charge_level
            if current_Q_B.loc[min_price_index_1, 'Price'] < current_Q_A.loc[max_price_index_1, 'Price']:
                  if min_price_index_1 < max_price_index_1:
                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                    charge_level += bottleneck_1
                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                    charge_level -= bottleneck_2
            
                    profit = (current_df.loc[max_price_index_1, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)/eff_2)
            
                    prices.append((min_price_index_1, current_df.loc[min_price_index_1, 'Price'], max_price_index_1, current_df.loc[max_price_index_1, 'Price'], profit, charge_level))
            
                  elif min_price_index_1 > max_price_index_1:
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
            
                        profit = (current_df.loc[max_price_index_1, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)/eff_2)
            
                        prices.append((min_price_index_1, current_df.loc[min_price_index_1, 'Price'], max_price_index_1, current_df.loc[max_price_index_1, 'Price'], profit, charge_level))
            
                  else:
                        continue  

                        
            smaller_index_1 = min(min_price_index_1, max_price_index_1)
            larger_index_1 = max(min_price_index_1, max_price_index_1)
            remaining_prices_01 = current_df[(current_df.index > smaller_index_1) & (current_df.index < larger_index_1)]
            remaining_prices_01_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_1) & (Q_A_Preds.index < larger_index_1)]
            remaining_prices_01_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_1) & (Q_B_Preds.index < larger_index_1)]

            if len(remaining_prices_01) > 1:
                max_price_index_4 = remaining_prices_01_A['Price'].idxmax()
                min_price_index_4 = remaining_prices_01_B['Price'].idxmin()
                if current_Q_B.loc[min_price_index_4, 'Price'] < current_Q_A.loc[max_price_index_4, 'Price']:
                    if min_price_index_4 < max_price_index_4:
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                    
                        profit = (current_df.loc[max_price_index_4, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_4, 'Price'] * bottleneck_1)/eff_2)
                    
                        prices.append((min_price_index_4, current_df.loc[min_price_index_4, 'Price'], max_price_index_1, current_df.loc[max_price_index_4, 'Price'], profit, charge_level))
                    
                    elif min_price_index_4 > max_price_index_4:
                          bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                          charge_level -= bottleneck_2
                          bottleneck_1 = min(capacity - charge_level, ramp_rate)
                          charge_level += bottleneck_1
                    
                          profit = (current_df.loc[max_price_index_4, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)/eff_2)
                    
                          prices.append((min_price_index_4, current_df.loc[min_price_index_4, 'Price'], max_price_index_4, current_df.loc[max_price_index_4, 'Price'], profit, charge_level))
                                
                    else:
                         continue

                  
        if len(remaining_prices_1) > 1:
                max_price_index_2 = remaining_prices_1_A['Price'].idxmax()
                min_price_index_2 = remaining_prices_1_B['Price'].idxmin()
                if current_Q_B.loc[min_price_index_2, 'Price'] < current_Q_A.loc[max_price_index_2, 'Price']:
                      if min_price_index_2 < max_price_index_2:
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
            
                        profit = (current_df.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_2, 'Price'] * bottleneck_1)/eff_2)
            
                        prices.append((min_price_index_2, current_df.loc[min_price_index_2, 'Price'], max_price_index_2, current_df.loc[max_price_index_2, 'Price'], profit, charge_level))
            
                      elif min_price_index_2 > max_price_index_2:
                         bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                         charge_level -= bottleneck_2
                         bottleneck_1 = min(capacity - charge_level, ramp_rate)
                         charge_level += bottleneck_1
            
                         profit = (current_df.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_2, 'Price'] * bottleneck_1)/eff_2)
            
                         prices.append((min_price_index_2, current_df.loc[min_price_index_2, 'Price'], max_price_index_2, current_df.loc[max_price_index_2, 'Price'], profit, charge_level))
            
                      else:
                            continue



                                            
                     

    return pd.DataFrame(prices, columns=['minPriceIndex', 'minPrice','maxPriceIndex', 'maxPrice', 'profit', 'charge Level'])

r_bm_50_50=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_50, Q_B_Preds=Q_50, eff_1=1, eff_2=1, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)
r_bm_10_90=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_10, Q_B_Preds=Q_90, eff_1=1, eff_2=1, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)
r_bm_30_70=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_30, Q_B_Preds=Q_70, eff_1=1, eff_2=1, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)
# r_bm_50_70=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_50, Q_B_Preds=Q_90, eff_1=1, eff_2=1, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)
# r_bm_70_90=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_70, Q_B_Preds=Q_90, eff_1=1, eff_2=1, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)


In [51]:
print(sum(r_bm_50_50.iloc[:,4:5].values))
r_bm_50_50

[52137.74]


,minPriceIndex,minPrice,maxPriceIndex,maxPrice,profit,charge Level
0,13,33.28,0,187.63,154.35,1
1,14,28.95,15,60.90,60.90,0
2,11,16.99,5,100.34,-16.99,1
3,31,14.21,22,45.66,31.45,1
4,30,14.47,25,44.76,30.29,1
...,...,...,...,...,...,...
2807,17488,46.40,17494,71.75,71.75,0
2808,17497,71.11,17499,58.45,-12.66,0
2809,17489,54.08,17493,64.43,10.35,0
2810,17504,42.38,17519,257.23,214.85,0


In [52]:
print(sum(r_bm_10_90.iloc[:,4:5].values))
r_bm_10_90

[168.79]


,minPriceIndex,minPrice,maxPriceIndex,maxPrice,profit,charge Level
0,10634,211.54,10624,380.33,168.79,1


In [53]:
print(sum(r_bm_30_70.iloc[:,4:5].values))
r_bm_30_70

[18499.68]


,minPriceIndex,minPrice,maxPriceIndex,maxPrice,profit,charge Level
0,925,0.00,912,7.48,7.48,1
1,924,0.00,913,6.34,6.34,1
2,1277,5.86,1269,2.01,-3.85,1
3,1337,0.00,1343,0.29,0.29,0
4,1344,13.76,1359,32.01,18.25,0
...,...,...,...,...,...,...
245,17262,83.43,17250,159.56,76.13,1
246,17312,74.89,17327,171.28,171.28,0
247,17359,68.06,17345,187.11,-68.06,1
248,17504,42.38,17519,257.23,257.23,0


In [54]:
num_rows = 365 * 48
df = pd.DataFrame(index=range(num_rows))

# Add the 'level_0' column
df['level_0'] = np.repeat(np.arange(366), 48)[:num_rows]

date_format="%m/%d/%Y %H:%M"
date_parse = lambda date: dt.datetime.strptime(date, date_format)
dat1 = pd.read_csv("/home/ciaran/rf_Q_1-12.csv")
dat1=pd.DataFrame(dat1)
dat1=pd.concat([dat1, df],axis=1)
column_names = ['lag_{}y_Forecast_10'.format(i) for i in range(2, 18)]
Q_10_BM=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_30'.format(i) for i in range(2, 18)]
Q_30_BM=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_50'.format(i) for i in range(2, 18)]
Q_50_BM=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_70'.format(i) for i in range(2, 18)]
Q_70_BM=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_90'.format(i) for i in range(2, 18)]
Q_90_BM=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y'.format(i) for i in range(2, 18)]
Y_r_BM=dat1[column_names].dropna().stack().reset_index()
# YY_test= Y_r.iloc[:, 2:]
Y_r_BM["Price"]=Y_r_BM.iloc[:,2:3]
Q_10_BM["Price"]=Q_10_BM.iloc[:,2:3]
Q_30_BM["Price"]=Q_30_BM.iloc[:,2:3]
Q_50_BM["Price"]=Q_50_BM.iloc[:,2:3]
Q_70_BM["Price"]=Q_70_BM.iloc[:,2:3]
Q_90_BM["Price"]=Q_90_BM.iloc[:,2:3]

Y_r_BM  = Y_r_BM.iloc[:,1:]
Q_10_BM = Q_10_BM.iloc[:,1:]
Q_30_BM = Q_30_BM.iloc[:,1:]
Q_50_BM = Q_50_BM.iloc[:,1:]
Q_70_BM = Q_70_BM.iloc[:,1:]
Q_90_BM = Q_90_BM.iloc[:,1:]

Y_r_BM= pd.concat([Y_r_BM, df],axis=1)
Q_10_BM= pd.concat([Q_10_BM, df],axis=1)
Q_30_BM= pd.concat([Q_30_BM, df],axis=1)
Q_50_BM= pd.concat([Q_50_BM, df],axis=1)
Q_70_BM= pd.concat([Q_70_BM, df],axis=1)
Q_90_BM= pd.concat([Q_90_BM, df],axis=1)

date_format="%d/%m/%Y %H:%M"
date_parse = lambda date: dt.datetime.strptime(date, date_format)
# dat = pd.read_csv("/home/ciaran/knn_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/lasso_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/lgbm_Q_DAM_1-12.csv")
dat = pd.read_csv("/home/ciaran/rf_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/SH_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/MH_Q_DAM_1-12.csv")

dat1=pd.DataFrame(dat)
start = 0
end = 17519
step = 2

values = list(range(start, end+1, step))
df1 = pd.DataFrame({'P_dam': values})

column_names = ['EURPrices+{}_Forecast_10'.format(i) for i in range(0, 24)]
Q_10_DAM=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_30'.format(i) for i in range(0, 24)]
Q_30_DAM=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_50'.format(i) for i in range(0, 24)]
Q_50_DAM=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_70'.format(i) for i in range(0, 24)]
Q_70_DAM=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_90'.format(i) for i in range(0, 24)]
Q_90_DAM=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}'.format(i) for i in range(0, 24)]
Y_r_DAM=dat1[column_names].dropna().stack().reset_index()
Y_r_DAM=Y_r_DAM.iloc[:,:]

Y_r_DAM["Price"]=Y_r_DAM.iloc[:,2:3]
Q_10_DAM["Price"]=Q_10_DAM.iloc[:,2:3]
Q_30_DAM["Price"]=Q_30_DAM.iloc[:,2:3]
Q_50_DAM["Price"]=Q_50_DAM.iloc[:,2:3]
Q_70_DAM["Price"]=Q_70_DAM.iloc[:,2:3]
Q_90_DAM["Price"]=Q_90_DAM.iloc[:,2:3]

Y_r_DAM= pd.concat([Y_r_DAM, df1], axis=1)
Q_10_DAM= pd.concat([Q_10_DAM, df1], axis=1)
Q_30_DAM= pd.concat([Q_30_DAM, df1], axis=1)
Q_50_DAM= pd.concat([Q_50_DAM, df1], axis=1)
Q_70_DAM= pd.concat([Q_70_DAM, df1], axis=1)
Q_90_DAM= pd.concat([Q_90_DAM, df1], axis=1)


In [55]:
def run_electricity_strategy(df, df_bm, Q_A_Preds, Q_B_Preds, Q_A_Preds_bm, Q_B_Preds_bm,  eff_1, eff_2, capacity,charge_level, ramp_rate, min_charge_level):
    prices = []
    charge_level = charge_level    
    level_0_values = df['level_0'].unique()

    for level_0 in level_0_values:
        current_df = df[df['level_0'] == level_0]
        current_Q_A = Q_A_Preds[Q_A_Preds['level_0'] == level_0]
        current_Q_B = Q_B_Preds[Q_B_Preds['level_0'] == level_0]
        current_df_bm = df_bm[df_bm['level_0'] == level_0]
        current_Q_A_bm = Q_A_Preds_bm[Q_A_Preds_bm['level_0'] == level_0]
        current_Q_B_bm = Q_B_Preds_bm[Q_B_Preds_bm['level_0'] == level_0]  
        
        max_price_index = current_Q_A['Price'].idxmax()
        min_price_index = current_Q_B['Price'].idxmin()
        
        max_price_index = current_Q_A.loc[max_price_index, 'P_dam']
        min_price_index = current_Q_B.loc[min_price_index, 'P_dam']        

        smaller_index = min(min_price_index, max_price_index)
        larger_index = max(min_price_index, max_price_index)
        
        
        remaining_prices_0 = current_df[(current_df['P_dam'] > smaller_index) & (current_df['P_dam'] < larger_index)]
        remaining_prices_0_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds['P_dam'] > smaller_index) & (Q_A_Preds['P_dam'] < larger_index)]
        remaining_prices_0_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds['P_dam'] > smaller_index) & (Q_B_Preds['P_dam'] < larger_index)]
        
        remaining_prices_1 = current_df_bm[(current_df_bm.index > smaller_index) & (current_df_bm.index > larger_index)]
        remaining_prices_1_A = Q_A_Preds_bm[(Q_A_Preds_bm['level_0'] == level_0) & (Q_A_Preds_bm.index > smaller_index) & (Q_A_Preds_bm.index > larger_index)]
        remaining_prices_1_B = Q_B_Preds_bm[(Q_B_Preds_bm['level_0'] == level_0) & (Q_B_Preds_bm.index > smaller_index) & (Q_B_Preds_bm.index > larger_index)]
        remaining_prices_2 = current_df_bm[(current_df_bm.index < smaller_index) & (current_df_bm.index < larger_index)]
        remaining_prices_2_A = Q_A_Preds_bm[(Q_A_Preds_bm['level_0'] == level_0) & (Q_A_Preds_bm.index < smaller_index) & (Q_A_Preds_bm.index < larger_index)]
        remaining_prices_2_B = Q_B_Preds_bm[(Q_B_Preds_bm['level_0'] == level_0) & (Q_B_Preds_bm.index < smaller_index) & (Q_B_Preds_bm.index < larger_index)]

        if current_Q_B.loc[min_price_index/2, 'Price'] < current_Q_A.loc[max_price_index/2, 'Price']:
            if min_price_index < max_price_index:
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
            
                profit = (current_df.loc[max_price_index/2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index/2, 'Price'] * bottleneck_1)/eff_2)
            
                prices.append((min_price_index, current_df.loc[min_price_index/2, 'Price'], max_price_index, current_df.loc[max_price_index/2, 'Price'], profit, charge_level))
            
            elif min_price_index > max_price_index:
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
            
                profit = (current_df.loc[max_price_index/2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index/2, 'Price'] * bottleneck_1)/eff_2)
            
                prices.append((min_price_index, current_df.loc[min_price_index/2, 'Price'],  max_price_index, current_df.loc[max_price_index/2, 'Price'], profit, charge_level))
            
            else:
                continue
                               
                        
        if len(remaining_prices_0) > 1:
            max_price_index_1 = remaining_prices_0_A['Price'].idxmax()
            min_price_index_1 = remaining_prices_0_B['Price'].idxmin()            
            max_price_index_1 = remaining_prices_0_A.loc[max_price_index_1, 'P_dam']
            min_price_index_1 = remaining_prices_0_B.loc[min_price_index_1, 'P_dam']
            
            charge_level=charge_level
            if current_Q_B.loc[min_price_index_1/2, 'Price'] < current_Q_A.loc[max_price_index_1/2, 'Price']:
                  if min_price_index_1 < max_price_index_1:
                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                    charge_level += bottleneck_1
                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                    charge_level -= bottleneck_2
            
                    profit = (current_df.loc[max_price_index_1/2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1/2, 'Price'] * bottleneck_1)/eff_2)
            
                    prices.append((min_price_index_1, current_df.loc[min_price_index_1/2, 'Price'], max_price_index_1, current_df.loc[max_price_index_1/2, 'Price'], profit, charge_level))
            
                  elif min_price_index_1 > max_price_index_1:
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
            
                        profit = (current_df.loc[max_price_index_1/2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1/2, 'Price'] * bottleneck_1)/eff_2)
            
                        prices.append((min_price_index_1, current_df.loc[min_price_index_1/2, 'Price'], max_price_index_1, current_df.loc[max_price_index_1/2, 'Price'], profit, charge_level))
            
                  else:
                        continue  
 
                

            smaller_index_1 = min(min_price_index_1, max_price_index_1)
            larger_index_1 = max(min_price_index_1, max_price_index_1)
            remaining_prices_01 = current_df[(current_df['P_dam'] > smaller_index_1) & (current_df['P_dam'] < larger_index_1)]
            remaining_prices_01_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds['P_dam'] > smaller_index_1) & (Q_A_Preds['P_dam'] < larger_index_1)]
            remaining_prices_01_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds['P_dam'] > smaller_index_1) & (Q_B_Preds['P_dam'] < larger_index_1)]

            if len(remaining_prices_01) > 1:

                    max_price_index_4 = remaining_prices_01_A['Price'].idxmax()
                    min_price_index_4 = remaining_prices_01_B['Price'].idxmin()
                    max_price_index_4 = remaining_prices_01_A.loc[max_price_index_4, 'P_dam']
                    min_price_index_4 = remaining_prices_01_B.loc[min_price_index_4, 'P_dam']
                    if current_Q_B.loc[min_price_index_4/2, 'Price'] < current_Q_A.loc[max_price_index_4/2, 'Price']:
                        if min_price_index_4 < max_price_index_4:
                            bottleneck_1 = min(capacity - charge_level, ramp_rate)
                            charge_level += bottleneck_1
                            bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                            charge_level -= bottleneck_2
                    
                            profit = (current_df.loc[max_price_index_4/2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_4/2, 'Price'] * bottleneck_1)/eff_2)
                    
                            prices.append((min_price_index_4, current_df.loc[min_price_index_4/2, 'Price'], max_price_index_4, current_df.loc[max_price_index_4/2, 'Price'], profit, charge_level))
                    
                        elif min_price_index_4 > max_price_index_4:
                              bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                              charge_level -= bottleneck_2
                              bottleneck_1 = min(capacity - charge_level, ramp_rate)
                              charge_level += bottleneck_1
                    
                              profit = (current_df.loc[max_price_index_4/2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1/2, 'Price'] * bottleneck_1)/eff_2)
                    
                              prices.append((min_price_index_4, current_df.loc[min_price_index_4/2, 'Price'], max_price_index_4, current_df.loc[max_price_index_4/2, 'Price'], profit, charge_level))
                                
                        else:
                             continue                     

                                            
        if len(remaining_prices_1) > 1:

                max_price_index_2 = remaining_prices_1_A['Price'].idxmax()
                min_price_index_2 = remaining_prices_1_B['Price'].idxmin()
                if current_Q_B_bm.loc[min_price_index_2, 'Price'] < current_Q_A_bm.loc[max_price_index_2, 'Price']:
                      if min_price_index_2 < max_price_index_2:
                            bottleneck_1 = min(capacity - charge_level, ramp_rate)
                            charge_level += bottleneck_1
                            bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                            charge_level -= bottleneck_2
            
                            profit = (current_df_bm.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_2, 'Price'] * bottleneck_1)/eff_2)
            
                            prices.append((min_price_index_2, current_df_bm.loc[min_price_index_2, 'Price'], max_price_index_2, current_df_bm.loc[max_price_index_2, 'Price'], profit, charge_level))
            
                      elif min_price_index_2 > max_price_index_2:
                             bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                             charge_level -= bottleneck_2
                             bottleneck_1 = min(capacity - charge_level, ramp_rate)
                             charge_level += bottleneck_1
            
                             profit = (current_df_bm.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_2, 'Price'] * bottleneck_1)/eff_2)
            
                             prices.append((min_price_index_2, current_df_bm.loc[min_price_index_2, 'Price'], max_price_index_2, current_df_bm.loc[max_price_index_2, 'Price'], profit, charge_level))
            
                      else:
                            continue
                        
                       
                        
                        
        if len(remaining_prices_2) > 1:
                max_price_index_3 = remaining_prices_2_A['Price'].idxmax()
                min_price_index_3 = remaining_prices_2_B['Price'].idxmin()
                if current_Q_B_bm.loc[min_price_index_3, 'Price'] < current_Q_A_bm.loc[max_price_index_3, 'Price']:
                      if min_price_index_3 < max_price_index_3:
                            bottleneck_1 = min(capacity - charge_level, ramp_rate)
                            charge_level += bottleneck_1
                            bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                            charge_level -= bottleneck_2
            
                            profit = (current_df_bm.loc[max_price_index_3, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_3, 'Price'] * bottleneck_1)/eff_2)
            
                            prices.append((min_price_index_3, current_df_bm.loc[min_price_index_3, 'Price'], max_price_index_3, current_df_bm.loc[max_price_index_3, 'Price'], profit, charge_level))
            
                      elif min_price_index_3 > max_price_index_3:
                            bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                            charge_level -= bottleneck_2
                            bottleneck_1 = min(capacity - charge_level, ramp_rate)
                            charge_level += bottleneck_1
            
                            profit = (current_df_bm.loc[max_price_index_3, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_3, 'Price'] * bottleneck_1)/eff_2)
            
                            prices.append((min_price_index_3, current_df_bm.loc[min_price_index_3, 'Price'], max_price_index_3, current_df_bm.loc[max_price_index_3, 'Price'], profit, charge_level))
            
                      else:
                            continue  
                        
                        
                      
                              

    return pd.DataFrame(prices, columns=['minPriceIndex', 'minPrice','maxPriceIndex', 'maxPrice', 'profit', 'charge Level'])



r_dam_bm_50_50=run_electricity_strategy(df=Y_r_DAM, df_bm=Y_r_BM, Q_A_Preds=Q_50_DAM, Q_B_Preds=Q_50_DAM, Q_A_Preds_bm=Q_50_BM, Q_B_Preds_bm=Q_50_BM, eff_1=1, eff_2=1, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)
r_dam_bm_10_90=run_electricity_strategy(df=Y_r_DAM, df_bm=Y_r_BM, Q_A_Preds=Q_10_DAM, Q_B_Preds=Q_90_DAM, Q_A_Preds_bm=Q_10_BM, Q_B_Preds_bm=Q_90_BM, eff_1=1, eff_2=1, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)
r_dam_bm_30_70=run_electricity_strategy(df=Y_r_DAM, df_bm=Y_r_BM, Q_A_Preds=Q_30_DAM, Q_B_Preds=Q_70_DAM, Q_A_Preds_bm=Q_30_BM, Q_B_Preds_bm=Q_70_BM, eff_1=1, eff_2=1, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)


In [56]:
print(sum(r_dam_bm_50_50.iloc[:,4:5].values))
r_dam_bm_50_50

[65564.63]


,minPriceIndex,minPrice,maxPriceIndex,maxPrice,profit,charge Level
0,10,12.26,34,31.66,31.66,0
1,28,15.16,20,25.67,-15.16,1
2,26,13.92,22,23.83,8.67,1
3,35,8.05,47,27.74,27.74,0
4,9,175.37,0,187.63,-175.37,1
...,...,...,...,...,...,...
1720,17429,74.32,17424,59.30,-74.32,1
1721,17480,81.63,17488,121.10,121.10,0
1722,17482,86.57,17486,108.00,21.43,0
1723,17489,54.08,17519,257.23,203.15,0


In [57]:
print(sum(r_dam_bm_10_90.iloc[:,4:5].values))
r_dam_bm_10_90

[43167.]


,minPriceIndex,minPrice,maxPriceIndex,maxPrice,profit,charge Level
0,50,12.64,84,36.00,36.00,0
1,52,12.88,82,39.82,26.94,0
2,54,10.26,68,20.66,10.40,0
3,100,16.23,132,37.30,21.07,0
4,104,21.21,130,37.16,15.95,0
...,...,...,...,...,...,...
809,17334,86.24,17344,246.11,159.87,0
810,17382,85.31,17392,252.37,167.06,0
811,17384,87.65,17390,258.87,171.22,0
812,17430,77.62,17462,117.03,39.41,0


In [58]:
print(sum(r_dam_bm_30_70.iloc[:,4:5].values))
r_dam_bm_30_70

[55327.22]


,minPriceIndex,minPrice,maxPriceIndex,maxPrice,profit,charge Level
0,8,11.58,36,28.66,28.66,0
1,28,15.16,34,31.66,16.50,0
2,30,24.69,32,31.66,6.97,0
3,50,12.64,84,36.00,23.36,0
4,52,12.88,82,39.82,26.94,0
...,...,...,...,...,...,...
1114,17432,77.62,17460,123.22,45.60,0
1115,17452,83.86,17458,121.35,37.49,0
1116,17480,81.63,17488,121.10,39.47,0
1117,17482,86.57,17486,108.00,21.43,0


In [ ]:
def run_electricity_strategy(df, df_bm, Q_A_Preds, Q_B_Preds, Q_A_Preds_bm, Q_B_Preds_bm,  eff_1, eff_2, capacity,charge_level, ramp_rate, min_charge_level):
    prices = []
    charge_level = charge_level    
    level_0_values = df['level_0'].unique()

    for level_0 in level_0_values:
        current_df = df[df['level_0'] == level_0]
        current_Q_A = Q_A_Preds[Q_A_Preds['level_0'] == level_0]
        current_Q_B = Q_B_Preds[Q_B_Preds['level_0'] == level_0]
        current_df_bm = df_bm[df_bm['level_0'] == level_0]
        current_Q_A_bm = Q_A_Preds_bm[Q_A_Preds_bm['level_0'] == level_0]
        current_Q_B_bm = Q_B_Preds_bm[Q_B_Preds_bm['level_0'] == level_0]  
        
        max_price_index = current_Q_A['Price'].idxmax()
        min_price_index = current_Q_B['Price'].idxmin()
        
        max_price_index = current_Q_A.loc[max_price_index, 'P_dam']
        min_price_index = current_Q_B.loc[min_price_index, 'P_dam']        

        smaller_index = min(min_price_index, max_price_index)
        larger_index = max(min_price_index, max_price_index)
        
        
        remaining_prices_0 = current_df[(current_df['P_dam'] > smaller_index) & (current_df['P_dam'] < larger_index)]
        remaining_prices_0_A = Q_A_Preds[(Q_A_Preds['P_dam'] > smaller_index) & (Q_A_Preds['P_dam'] < larger_index)]
        remaining_prices_0_B = Q_B_Preds[(Q_B_Preds['P_dam'] > smaller_index) & (Q_B_Preds['P_dam'] < larger_index)]
        
        remaining_prices_1 = current_df_bm[(current_df_bm.index > smaller_index) & (current_df_bm.index > larger_index)]
        remaining_prices_1_A = Q_A_Preds_bm[(Q_A_Preds_bm.index > smaller_index) & (Q_A_Preds_bm.index > larger_index)]
        remaining_prices_1_B = Q_B_Preds_bm[(Q_B_Preds_bm.index > smaller_index) & (Q_B_Preds_bm.index > larger_index)]
        remaining_prices_2 = current_df_bm[(current_df_bm.index < smaller_index) & (current_df_bm.index < larger_index)]
        remaining_prices_2_A = Q_A_Preds_bm[(Q_A_Preds_bm.index < smaller_index) & (Q_A_Preds_bm.index < larger_index)]
        remaining_prices_2_B = Q_B_Preds_bm[(Q_B_Preds_bm.index < smaller_index) & (Q_B_Preds_bm.index < larger_index)]

        if current_Q_B.loc[min_price_index/2, 'Price'] < current_Q_A.loc[max_price_index/2, 'Price']:
            if min_price_index < max_price_index:
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
            
                profit = (current_df.loc[max_price_index/2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index/2, 'Price'] * bottleneck_1)/eff_2)
            
                prices.append((min_price_index, current_df.loc[min_price_index/2, 'Price'], max_price_index, current_df.loc[max_price_index/2, 'Price'], profit, charge_level))
            
            elif min_price_index > max_price_index:
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
            
                profit = (current_df.loc[max_price_index/2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index/2, 'Price'] * bottleneck_1)/eff_2)
            
                prices.append((min_price_index, current_df.loc[min_price_index/2, 'Price'],  max_price_index, current_df.loc[max_price_index/2, 'Price'], profit, charge_level))
            
            else:
                continue
                
                
                
                                    
      
        if len(remaining_prices_1) > 1:
            try:

                max_price_index_2 = remaining_prices_1_A['Price'].idxmax()
                min_price_index_2 = remaining_prices_1_B['Price'].idxmin()
                if current_Q_B_bm.loc[min_price_index_2, 'Price'] < current_Q_A_bm.loc[max_price_index_2, 'Price']:
                      if min_price_index_2 < max_price_index_2:
                            bottleneck_1 = min(capacity - charge_level, ramp_rate)
                            charge_level += bottleneck_1
                            bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                            charge_level -= bottleneck_2
            
                            profit = (current_df_bm.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_2, 'Price'] * bottleneck_1)/eff_2)
            
                            prices.append((min_price_index_2, current_df_bm.loc[min_price_index_2, 'Price'], max_price_index_2, current_df_bm.loc[max_price_index_2, 'Price'], profit, charge_level))
            
                      elif min_price_index_2 > max_price_index_2:
                             bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                             charge_level -= bottleneck_2
                             bottleneck_1 = min(capacity - charge_level, ramp_rate)
                             charge_level += bottleneck_1
            
                             profit = (current_df_bm.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_2, 'Price'] * bottleneck_1)/eff_2)
            
                             prices.append((min_price_index_2, current_df_bm.loc[min_price_index_2, 'Price'], max_price_index_2, current_df_bm.loc[max_price_index_2, 'Price'], profit, charge_level))
            
                      else:
                            continue
                        
            except KeyError:
                continue                        
                        
                        
        if len(remaining_prices_2) > 1:
            try:
                max_price_index_3 = remaining_prices_2_A['Price'].idxmax()
                min_price_index_3 = remaining_prices_2_B['Price'].idxmin()
                if current_Q_B_bm.loc[min_price_index_3, 'Price'] < current_Q_A_bm.loc[max_price_index_3, 'Price']:
                      if min_price_index_3 < max_price_index_3:
                            bottleneck_1 = min(capacity - charge_level, ramp_rate)
                            charge_level += bottleneck_1
                            bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                            charge_level -= bottleneck_2
            
                            profit = (current_df_bm.loc[max_price_index_3, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_3, 'Price'] * bottleneck_1)/eff_2)
            
                            prices.append((min_price_index_3, current_df_bm.loc[min_price_index_3, 'Price'], max_price_index_3, current_df_bm.loc[max_price_index_3, 'Price'], profit, charge_level))
            
                      elif min_price_index_3 > max_price_index_3:
                            bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                            charge_level -= bottleneck_2
                            bottleneck_1 = min(capacity - charge_level, ramp_rate)
                            charge_level += bottleneck_1
            
                            profit = (current_df_bm.loc[max_price_index_3, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_3, 'Price'] * bottleneck_1)/eff_2)
            
                            prices.append((min_price_index_3, current_df_bm.loc[min_price_index_3, 'Price'], max_price_index_3, current_df_bm.loc[max_price_index_3, 'Price'], profit, charge_level))
            
                      else:
                            continue  
                        
                        
            except KeyError:
                continue                        
                        
                        
        if len(remaining_prices_0) > 1:
            max_price_index_1 = remaining_prices_0_A['Price'].idxmax()
            min_price_index_1 = remaining_prices_0_B['Price'].idxmin()            
            max_price_index_1 = remaining_prices_0_A.loc[max_price_index_1, 'P_dam']
            min_price_index_1 = remaining_prices_0_B.loc[min_price_index_1, 'P_dam']
            
            charge_level=charge_level
            if current_Q_B.loc[min_price_index_1/2, 'Price'] < current_Q_A.loc[max_price_index_1/2, 'Price']:
                  if min_price_index_1 < max_price_index_1:
                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                    charge_level += bottleneck_1
                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                    charge_level -= bottleneck_2
            
                    profit = (current_df.loc[max_price_index_1/2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1/2, 'Price'] * bottleneck_1)/eff_2)
            
                    prices.append((min_price_index_1, current_df.loc[min_price_index_1/2, 'Price'], max_price_index_1, current_df.loc[max_price_index_1/2, 'Price'], profit, charge_level))
            
                  elif min_price_index_1 > max_price_index_1:
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
            
                        profit = (current_df.loc[max_price_index_1/2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1/2, 'Price'] * bottleneck_1)/eff_2)
            
                        prices.append((min_price_index_1, current_df.loc[min_price_index_1/2, 'Price'], max_price_index_1, current_df.loc[max_price_index_1/2, 'Price'], profit, charge_level))
            
                  else:
                        continue  
 
                

            smaller_index_1 = min(min_price_index_1, max_price_index_1)
            larger_index_1 = max(min_price_index_1, max_price_index_1)
            remaining_prices_01 = current_df[(current_df['P_dam'] > smaller_index_1) & (current_df['P_dam'] < larger_index_1)]
            remaining_prices_01_A = Q_A_Preds[(Q_A_Preds['P_dam'] > smaller_index_1) & (Q_A_Preds['P_dam'] < larger_index_1)]
            remaining_prices_01_B = Q_B_Preds[(Q_B_Preds['P_dam'] > smaller_index_1) & (Q_B_Preds['P_dam'] < larger_index_1)]

            if len(remaining_prices_01) > 1:
                try:

                    max_price_index_4 = remaining_prices_01_A['Price'].idxmax()
                    min_price_index_4 = remaining_prices_01_B['Price'].idxmin()
                    max_price_index_4 = remaining_prices_01_A.loc[max_price_index_4, 'P_dam']
                    min_price_index_4 = remaining_prices_01_B.loc[min_price_index_4, 'P_dam']
                    if current_Q_B.loc[min_price_index_4/2, 'Price'] < current_Q_A.loc[max_price_index_4/2, 'Price']:
                        if min_price_index_4 < max_price_index_4:
                            bottleneck_1 = min(capacity - charge_level, ramp_rate)
                            charge_level += bottleneck_1
                            bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                            charge_level -= bottleneck_2
                    
                            profit = (current_df.loc[max_price_index_4/2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_4/2, 'Price'] * bottleneck_1)/eff_2)
                    
                            prices.append((min_price_index_4, current_df.loc[min_price_index_4, 'Price'], max_price_index_1, current_df.loc[max_price_index_4, 'Price'], profit, charge_level))
                    
                        elif min_price_index_4 > max_price_index_4:
                              bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                              charge_level -= bottleneck_2
                              bottleneck_1 = min(capacity - charge_level, ramp_rate)
                              charge_level += bottleneck_1
                    
                              profit = (current_df.loc[max_price_index_4/2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1/2, 'Price'] * bottleneck_1)/eff_2)
                    
                              prices.append((min_price_index_4, current_df.loc[min_price_index_4/2, 'Price'], max_price_index_4, current_df.loc[max_price_index_4/2, 'Price'], profit, charge_level))
                                
                        else:
                             continue                     
                except KeyError:
                     continue
                                            
                     

    return pd.DataFrame(prices, columns=['minPriceIndex', 'minPrice','maxPriceIndex', 'maxPrice', 'profit', 'charge Level'])



r_dam_bm_50_50=run_electricity_strategy(df=Y_r_DAM, df_bm=Y_r_BM, Q_A_Preds=Q_50_DAM, Q_B_Preds=Q_50_DAM, Q_A_Preds_bm=Q_50_BM, Q_B_Preds_bm=Q_50_BM, eff_1=1, eff_2=1, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)
r_dam_bm_10_90=run_electricity_strategy(df=Y_r_DAM, df_bm=Y_r_BM, Q_A_Preds=Q_10_DAM, Q_B_Preds=Q_90_DAM, Q_A_Preds_bm=Q_10_BM, Q_B_Preds_bm=Q_90_BM, eff_1=1, eff_2=1, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)
r_dam_bm_30_70=run_electricity_strategy(df=Y_r_DAM, df_bm=Y_r_BM, Q_A_Preds=Q_30_DAM, Q_B_Preds=Q_70_DAM, Q_A_Preds_bm=Q_30_BM, Q_B_Preds_bm=Q_70_BM, eff_1=1, eff_2=1, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)


In [ ]:
def run_electricity_strategy(df, Q_A_Preds, Q_B_Preds,  eff_1, eff_2, capacity,charge_level, ramp_rate, min_charge_level):
    prices = []
    charge_level = charge_level    
    level_0_values = df['level_0'].unique()

    for level_0 in level_0_values:
        current_df = df[df['level_0'] == level_0]
        current_Q_A = Q_A_Preds[Q_A_Preds['level_0'] == level_0]
        current_Q_B = Q_B_Preds[Q_B_Preds['level_0'] == level_0]
        
        max_price_index = current_Q_A['Price'].idxmax()
        min_price_index = current_Q_B['Price'].idxmin()

        smaller_index = min(min_price_index, max_price_index)
        larger_index = max(min_price_index, max_price_index)
        
        remaining_prices_0 = current_df[(current_df.index > smaller_index) & (current_df.index < larger_index)]
        remaining_prices_0_A = Q_A_Preds[(Q_A_Preds.index > smaller_index) & (Q_A_Preds.index < larger_index)]
        remaining_prices_0_B = Q_B_Preds[(Q_B_Preds.index > smaller_index) & (Q_B_Preds.index < larger_index)]
        
        remaining_prices_1 = current_df[(current_df.index > smaller_index) & (current_df.index > larger_index)]
        remaining_prices_1_A = Q_A_Preds[(Q_A_Preds.index > smaller_index) & (Q_A_Preds.index > larger_index)]
        remaining_prices_1_B = Q_B_Preds[(Q_B_Preds.index > smaller_index) & (Q_B_Preds.index > larger_index)]
        remaining_prices_2 = current_df[(current_df.index < smaller_index) & (current_df.index < larger_index)]
        remaining_prices_2_A = Q_A_Preds[(Q_A_Preds.index < smaller_index) & (Q_A_Preds.index < larger_index)]
        remaining_prices_2_B = Q_B_Preds[(Q_B_Preds.index < smaller_index) & (Q_B_Preds.index < larger_index)]

        if current_Q_B.loc[min_price_index, 'Price'] < current_Q_A.loc[max_price_index, 'Price']:
            if min_price_index < max_price_index:
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
            
                profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index, 'Price'] * bottleneck_1)/eff_2)
            
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'], max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))
            
            elif min_price_index > max_price_index:
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
            
                profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index, 'Price'] * bottleneck_1)/eff_2)
            
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'],  max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))
            
            else:
                continue
                                    
      
    
        if len(remaining_prices_0) > 1:
            max_price_index_1 = remaining_prices_0_A['Price'].idxmax()
            min_price_index_1 = remaining_prices_0_B['Price'].idxmin()
            charge_level=charge_level
            if current_Q_B.loc[min_price_index_1, 'Price'] < current_Q_A.loc[max_price_index_1, 'Price']:
                  if min_price_index_1 < max_price_index_1:
                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                    charge_level += bottleneck_1
                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                    charge_level -= bottleneck_2
            
                    profit = (current_df.loc[max_price_index_1, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)/eff_2)
            
                    prices.append((min_price_index_1, current_df.loc[min_price_index_1, 'Price'], max_price_index_1, current_df.loc[max_price_index_1, 'Price'], profit, charge_level))
            
                  elif min_price_index_1 > max_price_index_1:
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
            
                        profit = (current_df.loc[max_price_index_1, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)/eff_2)
            
                        prices.append((min_price_index_1, current_df.loc[min_price_index_1, 'Price'], max_price_index_1, current_df.loc[max_price_index_1, 'Price'], profit, charge_level))
            
                  else:
                        continue  
 
                    
            smaller_index_1 = min(min_price_index_1, max_price_index_1)
            larger_index_1 = max(min_price_index_1, max_price_index_1)
            remaining_prices_01 = current_df[(current_df.index > smaller_index_1) & (current_df.index < larger_index_1)]
            remaining_prices_01_A = Q_A_Preds[(Q_A_Preds.index > smaller_index_1) & (Q_A_Preds.index < larger_index_1)]
            remaining_prices_01_B = Q_B_Preds[(Q_B_Preds.index > smaller_index_1) & (Q_B_Preds.index < larger_index_1)]

            if len(remaining_prices_01) > 1:
                max_price_index_4 = remaining_prices_01_A['Price'].idxmax()
                min_price_index_4 = remaining_prices_01_B['Price'].idxmin()
                if current_Q_B.loc[min_price_index_4, 'Price'] < current_Q_A.loc[max_price_index_4, 'Price']:
                    if min_price_index_4 < max_price_index_4:
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                    
                        profit = (current_df.loc[max_price_index_4, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_4, 'Price'] * bottleneck_1)/eff_2)
                    
                        prices.append((min_price_index_4, current_df.loc[min_price_index_4, 'Price'], max_price_index_1, current_df.loc[max_price_index_4, 'Price'], profit, charge_level))
                    
                    elif min_price_index_4 > max_price_index_4:
                          bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                          charge_level -= bottleneck_2
                          bottleneck_1 = min(capacity - charge_level, ramp_rate)
                          charge_level += bottleneck_1
                    
                          profit = (current_df.loc[max_price_index_4, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)/eff_2)
                    
                          prices.append((min_price_index_4, current_df.loc[min_price_index_4, 'Price'], max_price_index_4, current_df.loc[max_price_index_4, 'Price'], profit, charge_level))
                                
                    else:
                         continue

                  
                    

                                            
                     

    return pd.DataFrame(prices, columns=['minPriceIndex', 'minPrice','maxPriceIndex', 'maxPrice', 'profit', 'charge Level'])



In [ ]:
        if len(remaining_prices_1) > 1:
            try:

                max_price_index_2 = remaining_prices_1_A['Price'].idxmax()
                min_price_index_2 = remaining_prices_1_B['Price'].idxmin()
                if current_Q_B.loc[min_price_index_2, 'Price'] < current_Q_A.loc[max_price_index_2, 'Price']:
                      if min_price_index_2 < max_price_index_2:
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
            
                        profit = (current_df.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_2, 'Price'] * bottleneck_1)/eff_2)
            
                        prices.append((min_price_index_2, current_df.loc[min_price_index_2, 'Price'], max_price_index_2, current_df.loc[max_price_index_2, 'Price'], profit, charge_level))
            
                      elif min_price_index_2 > max_price_index_2:
                         bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                         charge_level -= bottleneck_2
                         bottleneck_1 = min(capacity - charge_level, ramp_rate)
                         charge_level += bottleneck_1
            
                         profit = (current_df.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_2, 'Price'] * bottleneck_1)/eff_2)
            
                         prices.append((min_price_index_2, current_df.loc[min_price_index_2, 'Price'], max_price_index_2, current_df.loc[max_price_index_2, 'Price'], profit, charge_level))
            
                      else:
                            continue
            except KeyError:
                continue

In [ ]:
        if len(remaining_prices_1) > 1:
            try:

                max_price_index_2 = remaining_prices_1_A['Price'].idxmax()
                min_price_index_2 = remaining_prices_1_B['Price'].idxmin()
                if current_Q_B.loc[min_price_index_2, 'Price'] < current_Q_A.loc[max_price_index_2, 'Price']:
                      if min_price_index_2 < max_price_index_2:
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
            
                        profit = (current_df.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_2, 'Price'] * bottleneck_1)/eff_2)
            
                        prices.append((min_price_index_2, current_df.loc[min_price_index_2, 'Price'], max_price_index_2, current_df.loc[max_price_index_2, 'Price'], profit, charge_level))
            
                      elif min_price_index_2 > max_price_index_2:
                         bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                         charge_level -= bottleneck_2
                         bottleneck_1 = min(capacity - charge_level, ramp_rate)
                         charge_level += bottleneck_1
            
                         profit = (current_df.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_2, 'Price'] * bottleneck_1)/eff_2)
            
                         prices.append((min_price_index_2, current_df.loc[min_price_index_2, 'Price'], max_price_index_2, current_df.loc[max_price_index_2, 'Price'], profit, charge_level))
            
                      else:
                            continue

In [ ]:
def run_electricity_strategy(df, Q_A_Preds, Q_B_Preds,  eff_1, eff_2, capacity,charge_level, ramp_rate, min_charge_level):
    prices = []
    charge_level = charge_level    
    level_0_values = df['level_0'].unique()

    for level_0 in level_0_values:
        current_df = df[df['level_0'] == level_0]
        current_Q_A = Q_A_Preds[Q_A_Preds['level_0'] == level_0]
        current_Q_B = Q_B_Preds[Q_B_Preds['level_0'] == level_0]
        
        max_price_index = current_Q_A['Price'].idxmax()
        min_price_index = current_Q_B['Price'].idxmin()

        smaller_index = min(min_price_index, max_price_index)
        larger_index = max(min_price_index, max_price_index)
        
        remaining_prices_0 = current_df[(current_df.index > smaller_index) & (current_df.index < larger_index)]
        remaining_prices_0_A = Q_A_Preds[(Q_A_Preds.index > smaller_index) & (Q_A_Preds.index < larger_index)]
        remaining_prices_0_B = Q_B_Preds[(Q_B_Preds.index > smaller_index) & (Q_B_Preds.index < larger_index)]
        remaining_prices_1 = current_df[(current_df.index > smaller_index) & (current_df.index > larger_index)]
        remaining_prices_1_A = Q_A_Preds[(Q_A_Preds.index > smaller_index) & (Q_A_Preds.index > larger_index)]
        remaining_prices_1_B = Q_B_Preds[(Q_B_Preds.index > smaller_index) & (Q_B_Preds.index > larger_index)]
        remaining_prices_2 = current_df[(current_df.index < smaller_index) & (current_df.index < larger_index)]
        remaining_prices_2_A = Q_A_Preds[(Q_A_Preds.index < smaller_index) & (Q_A_Preds.index < larger_index)]
        remaining_prices_2_B = Q_B_Preds[(Q_B_Preds.index < smaller_index) & (Q_B_Preds.index < larger_index)]

        if current_Q_B.loc[min_price_index, 'Price'] < current_Q_A.loc[max_price_index, 'Price']:
            if min_price_index < max_price_index:
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
            
                profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index, 'Price'] * bottleneck_1)/eff_2)
            
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'], max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))
            
            elif min_price_index > max_price_index:
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
            
                profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index, 'Price'] * bottleneck_1)/eff_2)
            
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'],  max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))
            
            else:
                continue
                                    
      
    
        if len(remaining_prices_0) > 1:
            max_price_index_1 = remaining_prices_0_A['Price'].idxmax()
            min_price_index_1 = remaining_prices_0_B['Price'].idxmin()
            charge_level=charge_level
            if current_Q_B.loc[min_price_index_1, 'Price'] < current_Q_A.loc[max_price_index_1, 'Price']:
                  if min_price_index_1 < max_price_index_1:
                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                    charge_level += bottleneck_1
                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                    charge_level -= bottleneck_2
            
                    profit = (current_df.loc[max_price_index_1, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)/eff_2)
            
                    prices.append((min_price_index_1, current_df.loc[min_price_index_1, 'Price'], max_price_index_1, current_df.loc[max_price_index_1, 'Price'], profit, charge_level))
            
                  elif min_price_index_1 > max_price_index_1:
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
            
                        profit = (current_df.loc[max_price_index_1, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)/eff_2)
            
                        prices.append((min_price_index_1, current_df.loc[min_price_index_1, 'Price'], max_price_index_1, current_df.loc[max_price_index_1, 'Price'], profit, charge_level))
            
                  else:
                        continue  
 
                    
            smaller_index_1 = min(min_price_index_1, max_price_index_1)
            larger_index_1 = max(min_price_index_1, max_price_index_1)
            remaining_prices_01 = current_df[(current_df.index > smaller_index_1) & (current_df.index < larger_index_1)]
            remaining_prices_01_A = Q_A_Preds[(Q_A_Preds.index > smaller_index_1) & (Q_A_Preds.index < larger_index_1)]
            remaining_prices_01_B = Q_B_Preds[(Q_B_Preds.index > smaller_index_1) & (Q_B_Preds.index < larger_index_1)]

            if len(remaining_prices_01) > 1:
                max_price_index_4 = remaining_prices_01_A['Price'].idxmax()
                min_price_index_4 = remaining_prices_01_B['Price'].idxmin()
                if current_Q_B.loc[min_price_index_4, 'Price'] < current_Q_A.loc[max_price_index_4, 'Price']:
                    if min_price_index_4 < max_price_index_4:
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                    
                        profit = (current_df.loc[max_price_index_4, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_4, 'Price'] * bottleneck_1)/eff_2)
                    
                        prices.append((min_price_index_4, current_df.loc[min_price_index_4, 'Price'], max_price_index_1, current_df.loc[max_price_index_4, 'Price'], profit, charge_level))
                    
                    elif min_price_index_4 > max_price_index_4:
                          bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                          charge_level -= bottleneck_2
                          bottleneck_1 = min(capacity - charge_level, ramp_rate)
                          charge_level += bottleneck_1
                    
                          profit = (current_df.loc[max_price_index_4, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)/eff_2)
                    
                          prices.append((min_price_index_4, current_df.loc[min_price_index_4, 'Price'], max_price_index_4, current_df.loc[max_price_index_4, 'Price'], profit, charge_level))
                                
                    else:
                         continue

                  
                    
        if len(remaining_prices_1) > 1:
            max_price_index_2 = remaining_prices_1_A['Price'].idxmax()
            min_price_index_2 = remaining_prices_1_B['Price'].idxmin()
            if current_Q_B.loc[min_price_index_2, 'Price'] < current_Q_A.loc[max_price_index_2, 'Price']:
                  if min_price_index_2 < max_price_index_2:
                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                    charge_level += bottleneck_1
                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                    charge_level -= bottleneck_2
            
                    profit = (current_df.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_2, 'Price'] * bottleneck_1)/eff_2)
            
                    prices.append((min_price_index_2, current_df.loc[min_price_index_2, 'Price'], max_price_index_2, current_df.loc[max_price_index_2, 'Price'], profit, charge_level))
            
                  elif min_price_index_2 > max_price_index_2:
                     bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                     charge_level -= bottleneck_2
                     bottleneck_1 = min(capacity - charge_level, ramp_rate)
                     charge_level += bottleneck_1
            
                     profit = (current_df.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_2, 'Price'] * bottleneck_1)/eff_2)
            
                     prices.append((min_price_index_2, current_df.loc[min_price_index_2, 'Price'], max_price_index_2, current_df.loc[max_price_index_2, 'Price'], profit, charge_level))
            
                  else:
                        continue
                    

                    
                     

    return pd.DataFrame(prices, columns=['minPriceIndex', 'minPrice','maxPriceIndex', 'maxPrice', 'profit', 'charge Level'])


run_electricity_strategy(df=Y_r, Q_A_Preds=Q_10, Q_B_Preds=Q_90, eff_1=1, eff_2=1, capacity=4,charge_level=1, ramp_rate=1, min_charge_level=0)

In [ ]:
print(sum(run_electricity_strategy(df=Y_r, Q_A_Preds=Q_10, Q_B_Preds=Q_90, eff_1=1, eff_2=1, capacity=4,charge_level=1, ramp_rate=1, min_charge_level=0).iloc[:,4:5].values))

In [ ]:
#DAM

def run_electricity_strategy(df, Q_A_Preds, Q_B_Preds,  eff_1, eff_2, capacity,charge_level, ramp_rate, min_charge_level):
    prices = []
    charge_level = charge_level    
    level_0_values = df['level_0'].unique()

    for level_0 in level_0_values:
        current_df = df[df['level_0'] == level_0]
        current_Q_A = Q_A_Preds[Q_A_Preds['level_0'] == level_0]
        current_Q_B = Q_B_Preds[Q_B_Preds['level_0'] == level_0]
        
        max_price_index = current_Q_A['Price'].idxmax()
        min_price_index = current_Q_B['Price'].idxmin()

        smaller_index = min(min_price_index, max_price_index)
        larger_index = max(min_price_index, max_price_index)
        
        remaining_prices_0 = current_df[(current_df.index > smaller_index) & (current_df.index < larger_index)]
        remaining_prices_0_A = Q_A_Preds[(Q_A_Preds.index > smaller_index) & (Q_A_Preds.index < larger_index)]
        remaining_prices_0_B = Q_B_Preds[(Q_B_Preds.index > smaller_index) & (Q_B_Preds.index < larger_index)]
        remaining_prices_1 = current_df[(current_df.index > smaller_index) & (current_df.index > larger_index)]
        remaining_prices_1_A = Q_A_Preds[(Q_A_Preds.index > smaller_index) & (Q_A_Preds.index > larger_index)]
        remaining_prices_1_B = Q_B_Preds[(Q_B_Preds.index > smaller_index) & (Q_B_Preds.index > larger_index)]
        remaining_prices_2 = current_df[(current_df.index < smaller_index) & (current_df.index < larger_index)]
        remaining_prices_2_A = Q_A_Preds[(Q_A_Preds.index < smaller_index) & (Q_A_Preds.index < larger_index)]
        remaining_prices_2_B = Q_B_Preds[(Q_B_Preds.index < smaller_index) & (Q_B_Preds.index < larger_index)]

        if current_Q_B.loc[min_price_index, 'Price'] < current_Q_A.loc[max_price_index, 'Price']:
            if min_price_index < max_price_index:
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
            
                profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index, 'Price'] * bottleneck_1)/eff_2)
            
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'], max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))
            
            elif min_price_index > max_price_index:
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
            
                profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index, 'Price'] * bottleneck_1)/eff_2)
            
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'],  max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))
            
            else:
                continue
                                    
      
    
        if len(remaining_prices_0) > 1:
            max_price_index_1 = remaining_prices_0_A['Price'].idxmax()
            min_price_index_1 = remaining_prices_0_B['Price'].idxmin()
            charge_level=charge_level
            if current_Q_B.loc[min_price_index_1, 'Price'] < current_Q_A.loc[max_price_index_1, 'Price']:
                  if min_price_index_1 < max_price_index_1:
                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                    charge_level += bottleneck_1
                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                    charge_level -= bottleneck_2
            
                    profit = (current_df.loc[max_price_index_1, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)/eff_2)
            
                    prices.append((min_price_index_1, current_df.loc[min_price_index_1, 'Price'], max_price_index_1, current_df.loc[max_price_index_1, 'Price'], profit, charge_level))
            
                  elif min_price_index_1 > max_price_index_1:
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
            
                        profit = (current_df.loc[max_price_index_1, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)/eff_2)
            
                        prices.append((min_price_index_1, current_df.loc[min_price_index_1, 'Price'], max_price_index_1, current_df.loc[max_price_index_1, 'Price'], profit, charge_level))
            
                  else:
                        continue  
 
                    
            smaller_index_1 = min(min_price_index_1, max_price_index_1)
            larger_index_1 = max(min_price_index_1, max_price_index_1)
            remaining_prices_01 = current_df[(current_df.index > smaller_index_1) & (current_df.index < larger_index_1)]
            remaining_prices_01_A = Q_A_Preds[(Q_A_Preds.index > smaller_index_1) & (Q_A_Preds.index < larger_index_1)]
            remaining_prices_01_B = Q_B_Preds[(Q_B_Preds.index > smaller_index_1) & (Q_B_Preds.index < larger_index_1)]

            if len(remaining_prices_01) > 1:
                max_price_index_4 = remaining_prices_01_A['Price'].idxmax()
                min_price_index_4 = remaining_prices_01_B['Price'].idxmin()
                if current_Q_B.loc[min_price_index_4, 'Price'] < current_Q_A.loc[max_price_index_4, 'Price']:
                    if min_price_index_4 < max_price_index_4:
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                    
                        profit = (current_df.loc[max_price_index_4, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_4, 'Price'] * bottleneck_1)/eff_2)
                    
                        prices.append((min_price_index_4, current_df.loc[min_price_index_4, 'Price'], max_price_index_1, current_df.loc[max_price_index_4, 'Price'], profit, charge_level))
                    
                    elif min_price_index_4 > max_price_index_4:
                          bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                          charge_level -= bottleneck_2
                          bottleneck_1 = min(capacity - charge_level, ramp_rate)
                          charge_level += bottleneck_1
                    
                          profit = (current_df.loc[max_price_index_4, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)/eff_2)
                    
                          prices.append((min_price_index_4, current_df.loc[min_price_index_4, 'Price'], max_price_index_4, current_df.loc[max_price_index_4, 'Price'], profit, charge_level))
                                
                    else:
                         continue

                  
        if len(remaining_prices_1) > 1:
                max_price_index_2 = remaining_prices_1_A['Price'].idxmax()
                min_price_index_2 = remaining_prices_1_B['Price'].idxmin()
                if current_Q_B.loc[min_price_index_2, 'Price'] < current_Q_A.loc[max_price_index_2, 'Price']:
                      if min_price_index_2 < max_price_index_2:
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
            
                        profit = (current_df.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_2, 'Price'] * bottleneck_1)/eff_2)
            
                        prices.append((min_price_index_2, current_df.loc[min_price_index_2, 'Price'], max_price_index_2, current_df.loc[max_price_index_2, 'Price'], profit, charge_level))
            
                      elif min_price_index_2 > max_price_index_2:
                         bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                         charge_level -= bottleneck_2
                         bottleneck_1 = min(capacity - charge_level, ramp_rate)
                         charge_level += bottleneck_1
            
                         profit = (current_df.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_2, 'Price'] * bottleneck_1)/eff_2)
            
                         prices.append((min_price_index_2, current_df.loc[min_price_index_2, 'Price'], max_price_index_2, current_df.loc[max_price_index_2, 'Price'], profit, charge_level))
            
                      else:
                            continue


#         if len(remaining_prices_1) > 12:
# #             try:

#                 max_price_index_2 = remaining_prices_1_A['Price'].idxmax()
#                 min_price_index_2 = remaining_prices_1_B['Price'].idxmin()
#                 if current_Q_B.loc[min_price_index_2, 'Price'] < current_Q_A.loc[max_price_index_2, 'Price']:
#                       if min_price_index_2 < max_price_index_2:
#                         bottleneck_1 = min(capacity - charge_level, ramp_rate)
#                         charge_level += bottleneck_1
#                         bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
#                         charge_level -= bottleneck_2
            
#                         profit = (current_df.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_2, 'Price'] * bottleneck_1)/eff_2)
            
#                         prices.append((min_price_index_2, current_df.loc[min_price_index_2, 'Price'], max_price_index_2, current_df.loc[max_price_index_2, 'Price'], profit, charge_level))
            
#                       elif min_price_index_2 > max_price_index_2:
#                          bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
#                          charge_level -= bottleneck_2
#                          bottleneck_1 = min(capacity - charge_level, ramp_rate)
#                          charge_level += bottleneck_1
            
#                          profit = (current_df.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_2, 'Price'] * bottleneck_1)/eff_2)
            
#                          prices.append((min_price_index_2, current_df.loc[min_price_index_2, 'Price'], max_price_index_2, current_df.loc[max_price_index_2, 'Price'], profit, charge_level))
            
#                       else:
#                             continue
# #             except KeyError:
# #                 continue
                                            
                     

    return pd.DataFrame(prices, columns=['minPriceIndex', 'minPrice','maxPriceIndex', 'maxPrice', 'profit', 'charge Level'])



r_dam_50_50=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_50, Q_B_Preds=Q_50, eff_1=1, eff_2=1, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)
r_dam_10_90=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_10, Q_B_Preds=Q_90, eff_1=1, eff_2=1, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)
r_dam_30_70=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_30, Q_B_Preds=Q_70, eff_1=1, eff_2=1, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)

In [ ]:
def run_electricity_strategy(df, Q_A_Preds, Q_B_Preds,  eff_1, eff_2, capacity,charge_level, ramp_rate, min_charge_level):
    prices = []
    charge_level = charge_level    
    level_0_values = df['level_0'].unique()

    for level_0 in level_0_values:
        current_df = df[df['level_0'] == level_0]
        current_Q_A = Q_A_Preds[Q_A_Preds['level_0'] == level_0]
        current_Q_B = Q_B_Preds[Q_B_Preds['level_0'] == level_0]
        
        max_price_index = current_Q_A['Price'].idxmax()
        min_price_index = current_Q_B['Price'].idxmin()

        smaller_index = min(min_price_index, max_price_index)
        larger_index = max(min_price_index, max_price_index)
        
        remaining_prices_0 = current_df[(current_df.index > smaller_index) & (current_df.index < larger_index)]
        remaining_prices_0_A = Q_A_Preds[(Q_A_Preds.index > smaller_index) & (Q_A_Preds.index < larger_index)]
        remaining_prices_0_B = Q_B_Preds[(Q_B_Preds.index > smaller_index) & (Q_B_Preds.index < larger_index)]
        remaining_prices_1 = current_df[(current_df.index > smaller_index) & (current_df.index > larger_index)]
        remaining_prices_1_A = Q_A_Preds[(Q_A_Preds.index > smaller_index) & (Q_A_Preds.index > larger_index)]
        remaining_prices_1_B = Q_B_Preds[(Q_B_Preds.index > smaller_index) & (Q_B_Preds.index > larger_index)]
        remaining_prices_2 = current_df[(current_df.index < smaller_index) & (current_df.index < larger_index)]
        remaining_prices_2_A = Q_A_Preds[(Q_A_Preds.index < smaller_index) & (Q_A_Preds.index < larger_index)]
        remaining_prices_2_B = Q_B_Preds[(Q_B_Preds.index < smaller_index) & (Q_B_Preds.index < larger_index)]

        if current_Q_B.loc[min_price_index, 'Price'] < current_Q_A.loc[max_price_index, 'Price']:
            if min_price_index < max_price_index:
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
            
                profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index, 'Price'] * bottleneck_1)/eff_2)
            
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'], max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))
            
            elif min_price_index > max_price_index:
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
            
                profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index, 'Price'] * bottleneck_1)/eff_2)
            
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'],  max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))
            
            else:
                continue
                                    
      
    
        if len(remaining_prices_0) > 1:
            max_price_index_1 = remaining_prices_0_A['Price'].idxmax()
            min_price_index_1 = remaining_prices_0_B['Price'].idxmin()
            charge_level=charge_level
            if current_Q_B.loc[min_price_index_1, 'Price'] < current_Q_A.loc[max_price_index_1, 'Price']:
                  if min_price_index_1 < max_price_index_1:
                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                    charge_level += bottleneck_1
                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                    charge_level -= bottleneck_2
            
                    profit = (current_df.loc[max_price_index_1, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)/eff_2)
            
                    prices.append((min_price_index_1, current_df.loc[min_price_index_1, 'Price'], max_price_index_1, current_df.loc[max_price_index_1, 'Price'], profit, charge_level))
            
                  elif min_price_index_1 > max_price_index_1:
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
            
                        profit = (current_df.loc[max_price_index_1, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)/eff_2)
            
                        prices.append((min_price_index_1, current_df.loc[min_price_index_1, 'Price'], max_price_index_1, current_df.loc[max_price_index_1, 'Price'], profit, charge_level))
            
                  else:
                        continue  
 
                    
            smaller_index_1 = min(min_price_index_1, max_price_index_1)
            larger_index_1 = max(min_price_index_1, max_price_index_1)
            remaining_prices_01 = current_df[(current_df.index > smaller_index_1) & (current_df.index < larger_index_1)]
            remaining_prices_01_A = Q_A_Preds[(Q_A_Preds.index > smaller_index_1) & (Q_A_Preds.index < larger_index_1)]
            remaining_prices_01_B = Q_B_Preds[(Q_B_Preds.index > smaller_index_1) & (Q_B_Preds.index < larger_index_1)]

            if len(remaining_prices_01) > 1:
                max_price_index_4 = remaining_prices_01_A['Price'].idxmax()
                min_price_index_4 = remaining_prices_01_B['Price'].idxmin()
                if current_Q_B.loc[min_price_index_4, 'Price'] < current_Q_A.loc[max_price_index_4, 'Price']:
                    if min_price_index_4 < max_price_index_4:
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                    
                        profit = (current_df.loc[max_price_index_4, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_4, 'Price'] * bottleneck_1)/eff_2)
                    
                        prices.append((min_price_index_4, current_df.loc[min_price_index_4, 'Price'], max_price_index_1, current_df.loc[max_price_index_4, 'Price'], profit, charge_level))
                    
                    elif min_price_index_4 > max_price_index_4:
                          bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                          charge_level -= bottleneck_2
                          bottleneck_1 = min(capacity - charge_level, ramp_rate)
                          charge_level += bottleneck_1
                    
                          profit = (current_df.loc[max_price_index_4, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)/eff_2)
                    
                          prices.append((min_price_index_4, current_df.loc[min_price_index_4, 'Price'], max_price_index_4, current_df.loc[max_price_index_4, 'Price'], profit, charge_level))
                                
                    else:
                         continue

                  
                    
        if len(remaining_prices_1) > 1:
            max_price_index_2 = remaining_prices_1_A['Price'].idxmax()
            min_price_index_2 = remaining_prices_1_B['Price'].idxmin()
            if current_Q_B.loc[min_price_index_2, 'Price'] < current_Q_A.loc[max_price_index_2, 'Price']:
                  if min_price_index_2 < max_price_index_2:
                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                    charge_level += bottleneck_1
                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                    charge_level -= bottleneck_2
            
                    profit = (current_df.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_2, 'Price'] * bottleneck_1)/eff_2)
            
                    prices.append((min_price_index_2, current_df.loc[min_price_index_2, 'Price'], max_price_index_2, current_df.loc[max_price_index_2, 'Price'], profit, charge_level))
            
                  elif min_price_index_2 > max_price_index_2:
                     bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                     charge_level -= bottleneck_2
                     bottleneck_1 = min(capacity - charge_level, ramp_rate)
                     charge_level += bottleneck_1
            
                     profit = (current_df.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_2, 'Price'] * bottleneck_1)/eff_2)
            
                     prices.append((min_price_index_2, current_df.loc[min_price_index_2, 'Price'], max_price_index_2, current_df.loc[max_price_index_2, 'Price'], profit, charge_level))
            
                  else:
                        continue
                    

                    
                     

    return pd.DataFrame(prices, columns=['minPriceIndex', 'minPrice','maxPriceIndex', 'maxPrice', 'profit', 'charge Level'])


run_electricity_strategy(df=Y_r, Q_A_Preds=Q_10, Q_B_Preds=Q_90, eff_1=1, eff_2=1, capacity=4,charge_level=1, ramp_rate=1, min_charge_level=0)

In [ ]:
def run_electricity_strategy(df, Q_A_Preds, Q_B_Preds,  eff_1, eff_2, capacity,charge_level, ramp_rate, min_charge_level):
    prices = []
    charge_level = charge_level    
    level_0_values = df['level_0'].unique()

    for level_0 in level_0_values:
        current_df = df[df['level_0'] == level_0]
        current_Q_A = Q_A_Preds[Q_A_Preds['level_0'] == level_0]
        current_Q_B = Q_B_Preds[Q_B_Preds['level_0'] == level_0]
        
        max_price_index = current_Q_A['Price'].idxmax()
        min_price_index = current_Q_B['Price'].idxmin()

        smaller_index = min(min_price_index, max_price_index)
        larger_index = max(min_price_index, max_price_index)
        
        remaining_prices_0 = current_df[(current_df.index > smaller_index) & (current_df.index < larger_index)]
        remaining_prices_0_A = Q_A_Preds[(Q_A_Preds.index > smaller_index) & (Q_A_Preds.index < larger_index)]
        remaining_prices_0_B = Q_B_Preds[(Q_B_Preds.index > smaller_index) & (Q_B_Preds.index < larger_index)]
        remaining_prices_1 = current_df[(current_df.index > smaller_index) & (current_df.index > larger_index)]
        remaining_prices_1_A = Q_A_Preds[(Q_A_Preds.index > smaller_index) & (Q_A_Preds.index > larger_index)]
        remaining_prices_1_B = Q_B_Preds[(Q_B_Preds.index > smaller_index) & (Q_B_Preds.index > larger_index)]
        remaining_prices_2 = current_df[(current_df.index < smaller_index) & (current_df.index < larger_index)]
        remaining_prices_2_A = Q_A_Preds[(Q_A_Preds.index < smaller_index) & (Q_A_Preds.index < larger_index)]
        remaining_prices_2_B = Q_B_Preds[(Q_B_Preds.index < smaller_index) & (Q_B_Preds.index < larger_index)]

        if current_Q_B.loc[min_price_index, 'Price'] < current_Q_A.loc[max_price_index, 'Price']:
            if min_price_index < max_price_index:
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
            
                profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index, 'Price'] * bottleneck_1)/eff_2)
            
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'], max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))
            
            elif min_price_index > max_price_index:
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
            
                profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index, 'Price'] * bottleneck_1)/eff_2)
            
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'],  max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))
            
            else:
                continue
                                    
      
    
        if len(remaining_prices_0) > 1:
            max_price_index_1 = remaining_prices_0_A['Price'].idxmax()
            min_price_index_1 = remaining_prices_0_B['Price'].idxmin()
            charge_level=charge_level
            if current_Q_B.loc[min_price_index_1, 'Price'] < current_Q_A.loc[max_price_index_1, 'Price']:
                  if min_price_index_1 < max_price_index_1:
                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                    charge_level += bottleneck_1
                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                    charge_level -= bottleneck_2
            
                    profit = (current_df.loc[max_price_index_1, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)/eff_2)
            
                    prices.append((min_price_index_1, current_df.loc[min_price_index_1, 'Price'], max_price_index_1, current_df.loc[max_price_index_1, 'Price'], profit, charge_level))
            
                  elif min_price_index_1 > max_price_index_1:
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
            
                        profit = (current_df.loc[max_price_index_1, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)/eff_2)
            
                        prices.append((min_price_index_1, current_df.loc[min_price_index_1, 'Price'], max_price_index_1, current_df.loc[max_price_index_1, 'Price'], profit, charge_level))
            
                  else:
                        continue  
 
                    
            smaller_index_1 = min(min_price_index_1, max_price_index_1)
            larger_index_1 = max(min_price_index_1, max_price_index_1)
            remaining_prices_01 = current_df[(current_df.index > smaller_index_1) & (current_df.index < larger_index_1)]
            remaining_prices_01_A = Q_A_Preds[(Q_A_Preds.index > smaller_index_1) & (Q_A_Preds.index < larger_index_1)]
            remaining_prices_01_B = Q_B_Preds[(Q_B_Preds.index > smaller_index_1) & (Q_B_Preds.index < larger_index_1)]

            if len(remaining_prices_01) > 1:
                max_price_index_4 = remaining_prices_01_A['Price'].idxmax()
                min_price_index_4 = remaining_prices_01_B['Price'].idxmin()
                if current_Q_B.loc[min_price_index_4, 'Price'] < current_Q_A.loc[max_price_index_4, 'Price']:
                    if min_price_index_4 < max_price_index_4:
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                    
                        profit = (current_df.loc[max_price_index_4, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_4, 'Price'] * bottleneck_1)/eff_2)
                    
                        prices.append((min_price_index_4, current_df.loc[min_price_index_4, 'Price'], max_price_index_1, current_df.loc[max_price_index_4, 'Price'], profit, charge_level))
                    
                    elif min_price_index_4 > max_price_index_4:
                          bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                          charge_level -= bottleneck_2
                          bottleneck_1 = min(capacity - charge_level, ramp_rate)
                          charge_level += bottleneck_1
                    
                          profit = (current_df.loc[max_price_index_4, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)/eff_2)
                    
                          prices.append((min_price_index_4, current_df.loc[min_price_index_4, 'Price'], max_price_index_4, current_df.loc[max_price_index_4, 'Price'], profit, charge_level))
                                
                    else:
                         continue

                  
                    
        if len(remaining_prices_1) > 1:
            max_price_index_2 = remaining_prices_1_A['Price'].idxmax()
            min_price_index_2 = remaining_prices_1_B['Price'].idxmin()
            if current_Q_B.loc[min_price_index_2, 'Price'] < current_Q_A.loc[max_price_index_2, 'Price']:
                  if min_price_index_2 < max_price_index_2:
                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                    charge_level += bottleneck_1
                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                    charge_level -= bottleneck_2
            
                    profit = (current_df.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_2, 'Price'] * bottleneck_1)/eff_2)
            
                    prices.append((min_price_index_2, current_df.loc[min_price_index_2, 'Price'], max_price_index_2, current_df.loc[max_price_index_2, 'Price'], profit, charge_level))
            
                  elif min_price_index_2 > max_price_index_2:
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
            
                        profit = (current_df.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_2, 'Price'] * bottleneck_1)/eff_2)
            
                        prices.append((min_price_index_2, current_df.loc[min_price_index_2, 'Price'], max_price_index_2, current_df.loc[max_price_index_2, 'Price'], profit, charge_level))
            
                  else:
                        continue
                    

                    
                    
        if len(remaining_prices_2) > 1:
            max_price_index_3 = remaining_prices_2_A['Price'].idxmax()
            min_price_index_3 = remaining_prices_2_B['Price'].idxmin()
            if current_Q_A.loc[min_price_index_3, 'Price'] < current_Q_B.loc[max_price_index_3, 'Price']:
                  if min_price_index_3 < max_price_index_3:
                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                    charge_level += bottleneck_1
                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                    charge_level -= bottleneck_2
            
                    profit = (current_df.loc[max_price_index_3, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_3, 'Price'] * bottleneck_1)/eff_2)
            
                    prices.append((min_price_index_3, current_df.loc[min_price_index_3, 'Price'], max_price_index_3, current_df.loc[max_price_index_3, 'Price'], profit, charge_level))
            
                  elif min_price_index_3 > max_price_index_3:
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
            
                        profit = (current_df.loc[max_price_index_3, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_3, 'Price'] * bottleneck_1)/eff_2)
            
                        prices.append((min_price_index_3, current_df.loc[min_price_index_3, 'Price'], max_price_index_3, current_df.loc[max_price_index_3, 'Price'], profit, charge_level))
            
                  else:
                        continue         

    return pd.DataFrame(prices, columns=['minPriceIndex', 'minPrice','maxPriceIndex', 'maxPrice', 'profit', 'charge Level'])


run_electricity_strategy(df=Y_r, Q_A_Preds=Q_10, Q_B_Preds=Q_90, eff_1=1, eff_2=1, capacity=4,charge_level=1, ramp_rate=1, min_charge_level=0)

In [ ]:
def run_electricity_strategy(df, Q_A_Preds, Q_B_Preds,  eff_1, eff_2, capacity,charge_level, ramp_rate, min_charge_level):
    prices = []
    charge_level = charge_level    
    level_0_values = df['level_0'].unique()

    for level_0 in level_0_values:
        current_df  = df[df['level_0'] == level_0]
        current_Q_A = Q_A_Preds[Q_A_Preds['level_0'] == level_0]
        current_Q_B = Q_B_Preds[Q_B_Preds['level_0'] == level_0]

        max_price_index = current_Q_A['Price'].idxmax()
        min_price_index = current_Q_B['Price'].idxmin()

#         smaller_index = min(min_price_index, max_price_index)
#         larger_index = max(min_price_index, max_price_index)
        
#         remaining_prices_0 = current_df[(current_df.index > smaller_index) & (current_df.index < larger_index)]
#         remaining_prices_1 = current_df[(current_df.index > smaller_index) & (current_df.index > larger_index)]
#         remaining_prices_2 = current_df[(current_df.index < smaller_index) & (current_df.index < larger_index)]

        if current_Q_B.loc[min_price_index, 'Price'] < current_Q_A.loc[max_price_index, 'Price']:
            if min_price_index < max_price_index:
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
            
                profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index, 'Price'] * bottleneck_1)/eff_2)
            
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'], max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))
            
            elif min_price_index > max_price_index:
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
            
                profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index, 'Price'] * bottleneck_1)/eff_2)
            
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'],  max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))
            
            else:
                continue                   
   

    return pd.DataFrame(prices, columns=['minPriceIndex', 'minPrice','maxPriceIndex', 'maxPrice', 'profit', 'charge Level'])


run_electricity_strategy(df=Y_r, Q_A_Preds=Q_10, Q_B_Preds=Q_90, eff_1=1, eff_2=1, capacity=4,charge_level=1, ramp_rate=1, min_charge_level=0)

In [ ]:
print(sum(run_electricity_strategy(df=Y_r, Q_A_Preds=Q_10, Q_B_Preds=Q_90, eff_1=1, eff_2=1, capacity=4,charge_level=1, ramp_rate=1, min_charge_level=0).iloc[:,4:5].values))

In [ ]:
def run_electricity_strategy(df, eff, capacity,charge_level, ramp_rate, min_charge_level):
    prices = []
    charge_level = charge_level    
    level_0_values = df['level_0'].unique()

    for level_0 in level_0_values:
        current_df = df[df['level_0'] == level_0]
        max_price_index = current_df['Price'].idxmax()
        min_price_index = current_df['Price'].idxmin()

        smaller_index = min(min_price_index, max_price_index)
        larger_index = max(min_price_index, max_price_index)
        
        remaining_prices_0 = current_df[(current_df.index > smaller_index) & (current_df.index < larger_index)]
        remaining_prices_1 = current_df[(current_df.index > smaller_index) & (current_df.index > larger_index)]
        remaining_prices_2 = current_df[(current_df.index < smaller_index) & (current_df.index < larger_index)]

        if current_df.loc[min_price_index, 'Price'] < current_df.loc[max_price_index, 'Price'] * eff:
            if min_price_index < max_price_index:
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
            
                profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index, 'Price'] * bottleneck_1)
            
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'], max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))
            
            elif min_price_index > max_price_index:
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
            
                profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index, 'Price'] * bottleneck_1)
            
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'],  max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))
            
            else:
                continue
                                    
      
    
        if len(remaining_prices_0) > 1:
            max_price_index_1 = remaining_prices_0['Price'].idxmax()
            min_price_index_1 = remaining_prices_0['Price'].idxmin()
            charge_level=charge_level
            if current_df.loc[min_price_index_1, 'Price'] < current_df.loc[max_price_index_1, 'Price'] * eff:
                  if min_price_index_1 < max_price_index_1:
                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                    charge_level += bottleneck_1
                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                    charge_level -= bottleneck_2
            
                    profit = (current_df.loc[max_price_index_1, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)
            
                    prices.append((min_price_index_1, current_df.loc[min_price_index_1, 'Price'], max_price_index_1, current_df.loc[max_price_index_1, 'Price'], profit, charge_level))
            
                  elif min_price_index_1 > max_price_index_1:
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
            
                        profit = (current_df.loc[max_price_index_1, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)
            
                        prices.append((min_price_index_1, current_df.loc[min_price_index_1, 'Price'], max_price_index_1, current_df.loc[max_price_index_1, 'Price'], profit, charge_level))
            
                  else:
                        continue  
 
                    
            smaller_index_1 = min(min_price_index_1, max_price_index_1)
            larger_index_1 = max(min_price_index_1, max_price_index_1)
            remaining_prices_01 = current_df[(current_df.index > smaller_index_1) & (current_df.index < larger_index_1)]
            if len(remaining_prices_01) > 1:
                max_price_index_4 = remaining_prices_01['Price'].idxmax()
                min_price_index_4 = remaining_prices_01['Price'].idxmin()
                if current_df.loc[min_price_index_4, 'Price'] < current_df.loc[max_price_index_4, 'Price'] * eff:
                    if min_price_index_4 < max_price_index_4:
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                    
                        profit = (current_df.loc[max_price_index_4, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index_4, 'Price'] * bottleneck_1)
                    
                        prices.append((min_price_index_4, current_df.loc[min_price_index_4, 'Price'], max_price_index_1, current_df.loc[max_price_index_4, 'Price'], profit, charge_level))
                    
                    elif min_price_index_4 > max_price_index_4:
                          bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                          charge_level -= bottleneck_2
                          bottleneck_1 = min(capacity - charge_level, ramp_rate)
                          charge_level += bottleneck_1
                    
                          profit = (current_df.loc[max_price_index_4, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)
                    
                          prices.append((min_price_index_4, current_df.loc[min_price_index_4, 'Price'], max_price_index_4, current_df.loc[max_price_index_4, 'Price'], profit, charge_level))
                                
                    else:
                         continue

                    
                    
        if len(remaining_prices_1) > 1:
            max_price_index_2 = remaining_prices_1['Price'].idxmax()
            min_price_index_2 = remaining_prices_1['Price'].idxmin()
            if current_df.loc[min_price_index_2, 'Price'] < current_df.loc[max_price_index_2, 'Price'] * eff:
                  if min_price_index_2 < max_price_index_2:
                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                    charge_level += bottleneck_1
                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                    charge_level -= bottleneck_2
            
                    profit = (current_df.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index_2, 'Price'] * bottleneck_1)
            
                    prices.append((min_price_index_2, current_df.loc[min_price_index_2, 'Price'], max_price_index_2, current_df.loc[max_price_index_2, 'Price'], profit, charge_level))
            
                  elif min_price_index_2 > max_price_index_2:
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
            
                        profit = (current_df.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index_2, 'Price'] * bottleneck_1)
            
                        prices.append((min_price_index_2, current_df.loc[min_price_index_2, 'Price'], max_price_index_2, current_df.loc[max_price_index_2, 'Price'], profit, charge_level))
            
                  else:
                        continue
                    

                    
                    
        if len(remaining_prices_2) > 1:
            max_price_index_3 = remaining_prices_2['Price'].idxmax()
            min_price_index_3 = remaining_prices_2['Price'].idxmin()
            if current_df.loc[min_price_index_3, 'Price'] < current_df.loc[max_price_index_3, 'Price'] * eff:
                  if min_price_index_3 < max_price_index_3:
                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                    charge_level += bottleneck_1
                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                    charge_level -= bottleneck_2
            
                    profit = (current_df.loc[max_price_index_3, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index_3, 'Price'] * bottleneck_1)
            
                    prices.append((min_price_index_3, current_df.loc[min_price_index_3, 'Price'], max_price_index_3, current_df.loc[max_price_index_3, 'Price'], profit, charge_level))
            
                  elif min_price_index_3 > max_price_index_3:
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
            
                        profit = (current_df.loc[max_price_index_3, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index_3, 'Price'] * bottleneck_1)
            
                        prices.append((min_price_index_3, current_df.loc[min_price_index_3, 'Price'], max_price_index_3, current_df.loc[max_price_index_3, 'Price'], profit, charge_level))
            
                  else:
                        continue         

    return pd.DataFrame(prices, columns=['minPriceIndex', 'minPrice','maxPriceIndex', 'maxPrice', 'profit', 'charge Level'])


run_electricity_strategy(df=Y_r, eff=1, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)

In [ ]:
print(sum(run_electricity_strategy(df=Y_r, eff=1, capacity=4,charge_level=1, ramp_rate=1, min_charge_level=0).iloc[:,4:5].values))

In [ ]:
date_format="%m/%d/%Y %H:%M"
date_parse = lambda date: dt.datetime.strptime(date, date_format)
dat1 = pd.read_csv("/home/ciaran/SH_Q_1-12.csv")
dat1=pd.DataFrame(dat1)
column_names = ['lag_{}y_Forecast_50'.format(i) for i in range(2, 18)]
Q_50=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y'.format(i) for i in range(2, 18)]
Y_r=dat1[column_names].dropna().stack().reset_index()
# YY_test= Y_r.iloc[:, 2:]
Y_r["Price"]=Y_r.iloc[:,2:3]


In [ ]:
Y_r.head(n=40)

In [ ]:
def run_electricity_strategy(df, eff, capacity,charge_level, ramp_rate, min_charge_level):
    prices = []
    charge_level = charge_level    
    level_0_values = df['level_0'].unique()

    for level_0 in level_0_values:
        current_df = df[df['level_0'] == level_0]
        max_price_index = current_df['Price'].idxmax()
        min_price_index = current_df['Price'].idxmin()

        smaller_index = min(min_price_index, max_price_index)
        larger_index = max(min_price_index, max_price_index)
        
        remaining_prices_0 = current_df[(current_df.index > smaller_index) & (current_df.index < larger_index)]
        remaining_prices_1 = current_df[(current_df.index > smaller_index) & (current_df.index > larger_index)]
        remaining_prices_2 = current_df[(current_df.index < smaller_index) & (current_df.index < larger_index)]

        if current_df.loc[min_price_index, 'Price'] < current_df.loc[max_price_index, 'Price'] * eff:
            if min_price_index < max_price_index:
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
            
                profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index, 'Price'] * bottleneck_1)
            
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'], max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))
            
            elif min_price_index > max_price_index:
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
            
                profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index, 'Price'] * bottleneck_1)
            
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'],  max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))
            
            else:
                continue
                                    
      
    
        if len(remaining_prices_0) > 1:
            max_price_index_1 = remaining_prices_0['Price'].idxmax()
            min_price_index_1 = remaining_prices_0['Price'].idxmin()
            charge_level=charge_level
            if current_df.loc[min_price_index_1, 'Price'] < current_df.loc[max_price_index_1, 'Price'] * eff:
                  if min_price_index_1 < max_price_index_1:
                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                    charge_level += bottleneck_1
                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                    charge_level -= bottleneck_2
            
                    profit = (current_df.loc[max_price_index_1, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)
            
                    prices.append((min_price_index_1, current_df.loc[min_price_index_1, 'Price'], max_price_index_1, current_df.loc[max_price_index_1, 'Price'], profit, charge_level))
            
                  elif min_price_index_1 > max_price_index_1:
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
            
                        profit = (current_df.loc[max_price_index_1, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)
            
                        prices.append((min_price_index_1, current_df.loc[min_price_index_1, 'Price'], max_price_index_1, current_df.loc[max_price_index_1, 'Price'], profit, charge_level))
            
                  else:
                        continue  
 
                    
            smaller_index_1 = min(min_price_index_1, max_price_index_1)
            larger_index_1 = max(min_price_index_1, max_price_index_1)
            remaining_prices_01 = current_df[(current_df.index > smaller_index_1) & (current_df.index < larger_index_1)]
            if len(remaining_prices_01) > 1:
                max_price_index_4 = remaining_prices_01['Price'].idxmax()
                min_price_index_4 = remaining_prices_01['Price'].idxmin()
                if current_df.loc[min_price_index_4, 'Price'] < current_df.loc[max_price_index_4, 'Price'] * eff:
                    if min_price_index_4 < max_price_index_4:
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                    
                        profit = (current_df.loc[max_price_index_4, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index_4, 'Price'] * bottleneck_1)
                    
                        prices.append((min_price_index_4, current_df.loc[min_price_index_4, 'Price'], max_price_index_1, current_df.loc[max_price_index_4, 'Price'], profit, charge_level))
                    
                    elif min_price_index_4 > max_price_index_4:
                          bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                          charge_level -= bottleneck_2
                          bottleneck_1 = min(capacity - charge_level, ramp_rate)
                          charge_level += bottleneck_1
                    
                          profit = (current_df.loc[max_price_index_4, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)
                    
                          prices.append((min_price_index_4, current_df.loc[min_price_index_4, 'Price'], max_price_index_4, current_df.loc[max_price_index_4, 'Price'], profit, charge_level))
                                
                    else:
                         continue

                    
                    
        if len(remaining_prices_1) > 1:
            max_price_index_2 = remaining_prices_1['Price'].idxmax()
            min_price_index_2 = remaining_prices_1['Price'].idxmin()
            if current_df.loc[min_price_index_2, 'Price'] < current_df.loc[max_price_index_2, 'Price'] * eff:
                  if min_price_index_2 < max_price_index_2:
                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                    charge_level += bottleneck_1
                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                    charge_level -= bottleneck_2
            
                    profit = (current_df.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index_2, 'Price'] * bottleneck_1)
            
                    prices.append((min_price_index_2, current_df.loc[min_price_index_2, 'Price'], max_price_index_2, current_df.loc[max_price_index_2, 'Price'], profit, charge_level))
            
                  elif min_price_index_2 > max_price_index_2:
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
            
                        profit = (current_df.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index_2, 'Price'] * bottleneck_1)
            
                        prices.append((min_price_index_2, current_df.loc[min_price_index_2, 'Price'], max_price_index_2, current_df.loc[max_price_index_2, 'Price'], profit, charge_level))
            
                  else:
                        continue
                    

                    
                    
        if len(remaining_prices_2) > 1:
            max_price_index_3 = remaining_prices_2['Price'].idxmax()
            min_price_index_3 = remaining_prices_2['Price'].idxmin()
            if current_df.loc[min_price_index_3, 'Price'] < current_df.loc[max_price_index_3, 'Price'] * eff:
                  if min_price_index_3 < max_price_index_3:
                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                    charge_level += bottleneck_1
                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                    charge_level -= bottleneck_2
            
                    profit = (current_df.loc[max_price_index_3, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index_3, 'Price'] * bottleneck_1)
            
                    prices.append((min_price_index_3, current_df.loc[min_price_index_3, 'Price'], max_price_index_3, current_df.loc[max_price_index_3, 'Price'], profit, charge_level))
            
                  elif min_price_index_3 > max_price_index_3:
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
            
                        profit = (current_df.loc[max_price_index_3, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index_3, 'Price'] * bottleneck_1)
            
                        prices.append((min_price_index_3, current_df.loc[min_price_index_3, 'Price'], max_price_index_3, current_df.loc[max_price_index_3, 'Price'], profit, charge_level))
            
                  else:
                        continue         

    return pd.DataFrame(prices, columns=['minPriceIndex', 'minPrice','maxPriceIndex', 'maxPrice', 'profit', 'charge Level'])


run_electricity_strategy(df=Y_r, eff=1, capacity=10,charge_level=4, ramp_rate=4, min_charge_level=1)

In [ ]:
print(sum(run_electricity_strategy(df=Y_r, eff=1, capacity=10,charge_level=10, ramp_rate=10, min_charge_level=0).iloc[:,4:5].values))

In [ ]:
plt.plot(run_electricity_strategy(df=Y_r, eff=1, capacity=10,charge_level=10, ramp_rate=10, min_charge_level=0).iloc[:,4:5].values)

In [ ]:
def run_electricity_strategy(df, eff, capacity, ramp_rate, min_charge_level):
    prices = []
    charge_level = 1    
    level_0_values = df['level_0'].unique()

    for level_0 in level_0_values:
        current_df = df[df['level_0'] == level_0]
        max_price_index = current_df['Price'].idxmax()
        min_price_index = current_df['Price'].idxmin()

        smaller_index = min(min_price_index, max_price_index)
        larger_index = max(min_price_index, max_price_index)
        
        remaining_prices_0 = current_df[(current_df.index > smaller_index) & (current_df.index < larger_index)]
        remaining_prices_1 = current_df[(current_df.index > smaller_index) & (current_df.index > larger_index)]
        remaining_prices_2 = current_df[(current_df.index < smaller_index) & (current_df.index < larger_index)]

        if current_df.loc[min_price_index, 'Price'] < current_df.loc[max_price_index, 'Price'] * eff:
            if min_price_index < max_price_index:
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
            
                profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index, 'Price'] * bottleneck_1)
            
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'], max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))
            
            elif min_price_index > max_price_index:
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
            
                profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index, 'Price'] * bottleneck_1)
            
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'],  max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))
            
            else:
                continue
                                    
      
    
        if len(remaining_prices_0) > 1:
            max_price_index_1 = remaining_prices_0['Price'].idxmax()
            min_price_index_1 = remaining_prices_0['Price'].idxmin()
            if current_df.loc[min_price_index_1, 'Price'] < current_df.loc[max_price_index_1, 'Price'] * eff:
                  if min_price_index_1 < max_price_index_1:
                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                    charge_level += bottleneck_1
                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                    charge_level -= bottleneck_2
            
                    profit = (current_df.loc[max_price_index_1, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)
            
                    prices.append((min_price_index_1, current_df.loc[min_price_index_1, 'Price'], max_price_index_1, current_df.loc[max_price_index_1, 'Price'], profit, charge_level))
            
                  elif min_price_index_1 > max_price_index_1:
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
            
                        profit = (current_df.loc[max_price_index_1, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)
            
                        prices.append((min_price_index_1, current_df.loc[min_price_index_1, 'Price'], max_price_index_1, current_df.loc[max_price_index_1, 'Price'], profit, charge_level))
            
                  else:
                        continue  
       

    return pd.DataFrame(prices, columns=['minPriceIndex', 'minPrice','maxPriceIndex', 'maxPrice', 'profit', 'charge Level'])


run_electricity_strategy(df=Y_r, eff=1, capacity=5, ramp_rate=2, min_charge_level=1)

In [ ]:
def run_electricity_strategy(df, eff, capacity, ramp_rate, min_charge_level):
    prices = []
    charge_level = 5
    
    level_0_values = df['level_0'].unique()

    for level_0 in level_0_values:
        current_df = df[df['level_0'] == level_0]
        max_price_index = current_df['Price'].idxmax()
        min_price_index = current_df['Price'].idxmin()

        smaller_index = min(min_price_index, max_price_index)
        larger_index = max(min_price_index, max_price_index)
        
        if min_price_index < max_price_index:
            bottleneck_1 = min(capacity - charge_level, ramp_rate)
            bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
            charge_level += bottleneck_1
            charge_level -= bottleneck_2
            
            profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index, 'Price'] * bottleneck_1)
            
            prices.append((min_price_index, current_df.loc[min_price_index, 'Price'], 
                           max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))
            
        elif min_price_index > max_price_index:
            bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
            charge_level -= bottleneck_2
            bottleneck_1 = min(capacity - charge_level, ramp_rate)
            charge_level += bottleneck_1
            
            profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index, 'Price'] * bottleneck_1)
            
            prices.append((min_price_index, current_df.loc[min_price_index, 'Price'], 
                           max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))
            
        else:
            continue

    return prices
run_electricity_strategy(df=Y_r, eff=1, capacity=5, ramp_rate=2, min_charge_level=1)

In [ ]:
def run_electricity_strategy(df, eff, capacity, ramp_rate, min_charge_level):
    prices = []
    charge_level = 5

    level_0_values = df['level_0'].unique()

    for level_0 in level_0_values:
        current_df = df[df['level_0'] == level_0]
        max_price_index = current_df['Price'].idxmax()
        min_price_index = current_df['Price'].idxmin()

        smaller_index = min(min_price_index, max_price_index)
        larger_index = max(min_price_index, max_price_index)
        remaining_prices_0 = current_df[(current_df.index > smaller_index) & (current_df.index < larger_index)]

        if len(remaining_prices_0) > 1:
            if min_price_index < max_price_index:
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
                max_price = current_df.loc[max_price_index, 'Price']
                min_price = current_df.loc[min_price_index, 'Price']
                profit = (max_price * bottleneck_2 * eff) - (min_price * bottleneck_1)
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'], max_price_index, max_price, bottleneck_1, bottleneck_2, profit, charge_level))
                
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
                max_price_1 = remaining_prices_0['Price'].max()
                min_price_1 = remaining_prices_0['Price'].min()
                profit_1 = (max_price_1 * bottleneck_2 * eff) - (min_price_1 * bottleneck_1)
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'], remaining_prices_0['Price'].idxmax(), max_price_1, bottleneck_1, bottleneck_2, profit_1, charge_level))
                
            else:
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
                max_price_1 = remaining_prices_0['Price'].max()
                min_price_1 = remaining_prices_0['Price'].min()
                profit_1 = (max_price_1 * bottleneck_2 * eff) - (min_price_1 * bottleneck_1)
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'], max_price_index, current_df.loc[max_price_index, 'Price'], bottleneck_1, bottleneck_2, profit_1, charge_level))
                
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
                max_price = current_df.loc[max_price_index, 'Price']
                min_price = current_df.loc[min_price_index, 'Price']
                profit = (max_price * bottleneck_2 * eff) - (min_price * bottleneck_1)
                prices.append((remaining_prices_0['Price'].idxmin(), remaining_prices_0['Price'].min(), max_price_index, current_df.loc[max_price


run_electricity_strategy(df=Y_r, eff=1, capacity=5, ramp_rate=2, min_charge_level=1)

In [ ]:

profitSpecific(maxHourIndex)=(price(maxHourIndex)*eff-price(minHourIndex)); 
profitSpecific(minHourIndex)=-profitSpecific(maxHourIndex)

if price(maxHourIndex)*eff> price(minHourIndex)
    if minHourIndex<maxHourIndex 
        storageFree=capacity-max(chargeLevel(minHourIndex:maxHourIndex));
        bottleneck=min([TAvail(maxHourIndex),PAvail(minHourIndex),storageFree/periodT]); 
            
    else if minHourIndex>maxHourIndex 
        storageLeft=min(chargeLevel(maxHourIndex:minHourIndex));
        bottleneck=min([TAvail(maxHourIndex),PAvail(minHourIndex),storageLeft/periodT]); 
    end
        
        
TAvail(maxHourIndex)=TAvail(maxHourIndex)-bottleneck;   
PAvail(minHourIndex)=PAvail(minHourIndex)-bottleneck;   
Tuse(maxHourIndex)=Tuse(maxHourIndex)+bottleneck;       
Puse(minHourIndex)=Puse(minHourIndex)+bottleneck;      
        
chargeLevel(maxHourIndex+1:end)=chargeLevel(maxHourIndex+1:end)-bottleneck*periodT;
chargeLevel(minHourIndex+1:end)=chargeLevel(minHourIndex+1:end)+bottleneck*periodT;

profit(maxHourIndex)=Tuse(maxHourIndex)*periodT*profitSpecific(maxHourIndex);  
profit(minHourIndex)=-profit(maxHourIndex);

profitTotal=profitTotal+profit(maxHourIndex);


In [ ]:
def run_electricity_strategy(df, eff, capacity, ramp_rate, min_charge_level):
    prices = []
    charge_level = 5    
    level_0_values = df['level_0'].unique()

    for level_0 in level_0_values:
        current_df = df[df['level_0'] == level_0]
        max_price_index = current_df['Price'].idxmax()
        min_price_index = current_df['Price'].idxmin()

        smaller_index = min(min_price_index, max_price_index)
        larger_index = max(min_price_index, max_price_index)
        
        remaining_prices_0 = current_df[(current_df.index > smaller_index) & (current_df.index < larger_index)]
        remaining_prices_1 = current_df[(current_df.index > smaller_index) & (current_df.index > larger_index)]
        remaining_prices_2 = current_df[(current_df.index < smaller_index) & (current_df.index < larger_index)]

        if current_df.loc[min_price_index, 'Price'] < current_df.loc[max_price_index, 'Price'] * eff:
            if min_price_index < max_price_index:
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
            
                profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index, 'Price'] * bottleneck_1)
            
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'], max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))
            
            elif min_price_index > max_price_index:
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
            
                profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index, 'Price'] * bottleneck_1)
            
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'],  max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))
            
            else:
                continue
            
            
            
            
           
            
            
            
            
            
      
    
        if len(remaining_prices_0) > 1:
            max_price_index_1 = remaining_prices_0['Price'].idxmax()
            min_price_index_1 = remaining_prices_0['Price'].idxmin()
            if current_df.loc[min_price_index_1, 'Price'] < current_df.loc[max_price_index_1, 'Price'] * eff:
                  if min_price_index_1 < max_price_index_1:
                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                    charge_level += bottleneck_1
                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                    charge_level -= bottleneck_2
            
                    profit = (current_df.loc[max_price_index_1, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)
            
                    prices.append((min_price_index_1, current_df.loc[min_price_index_1, 'Price'], max_price_index_1, current_df.loc[max_price_index_1, 'Price'], profit, charge_level))
            
                  elif min_price_index_1 > max_price_index_1:
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
            
                        profit = (current_df.loc[max_price_index_1, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)
            
                        prices.append((min_price_index_1, current_df.loc[min_price_index_1, 'Price'], max_price_index_1, current_df.loc[max_price_index_1, 'Price'], profit, charge_level))
            
                  else:
                        continue
                    
                    
                    
 
                    
            smaller_index_1 = min(min_price_index_1, max_price_index_1)
            larger_index_1 = max(min_price_index_1, max_price_index_1)
            remaining_prices_01 = current_df[(current_df.index > smaller_index_1) & (current_df.index < larger_index_1)]
            if len(remaining_prices_01) > 1:
                max_price_index_4 = remaining_prices_01['Price'].idxmax()
                min_price_index_4 = remaining_prices_01['Price'].idxmin()
                if current_df.loc[min_price_index_4, 'Price'] < current_df.loc[max_price_index_4, 'Price'] * eff:
                    if min_price_index_4 < max_price_index_4:
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                    
                        profit = (current_df.loc[max_price_index_4, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index_4, 'Price'] * bottleneck_1)
                    
                        prices.append((min_price_index_4, current_df.loc[min_price_index_4, 'Price'], max_price_index_1, current_df.loc[max_price_index_4, 'Price'], profit, charge_level))
                    
                    elif min_price_index_4 > max_price_index_4:
                          bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                          charge_level -= bottleneck_2
                          bottleneck_1 = min(capacity - charge_level, ramp_rate)
                          charge_level += bottleneck_1
                    
                          profit = (current_df.loc[max_price_index_4, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)
                    
                          prices.append((min_price_index_4, current_df.loc[min_price_index_4, 'Price'], max_price_index_4, current_df.loc[max_price_index_4, 'Price'], profit, charge_level))
                                
                    else:
                         continue

                    
                    
        if len(remaining_prices_1) > 1:
            max_price_index_2 = remaining_prices_1['Price'].idxmax()
            min_price_index_2 = remaining_prices_1['Price'].idxmin()
            if current_df.loc[min_price_index_2, 'Price'] < current_df.loc[max_price_index_2, 'Price'] * eff:
                  if min_price_index_2 < max_price_index_2:
                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                    charge_level += bottleneck_1
                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                    charge_level -= bottleneck_2
            
                    profit = (current_df.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index_2, 'Price'] * bottleneck_1)
            
                    prices.append((min_price_index_2, current_df.loc[min_price_index_2, 'Price'], max_price_index_2, current_df.loc[max_price_index_2, 'Price'], profit, charge_level))
            
                  elif min_price_index_2 > max_price_index_2:
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
            
                        profit = (current_df.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index_2, 'Price'] * bottleneck_1)
            
                        prices.append((min_price_index_2, current_df.loc[min_price_index_2, 'Price'], max_price_index_2, current_df.loc[max_price_index_2, 'Price'], profit, charge_level))
            
                  else:
                        continue
                    

                    
                    
        if len(remaining_prices_2) > 1:
            max_price_index_3 = remaining_prices_2['Price'].idxmax()
            min_price_index_3 = remaining_prices_2['Price'].idxmin()
            if current_df.loc[min_price_index_3, 'Price'] < current_df.loc[max_price_index_3, 'Price'] * eff:
                  if min_price_index_3 < max_price_index_3:
                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                    charge_level += bottleneck_1
                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                    charge_level -= bottleneck_2
            
                    profit = (current_df.loc[max_price_index_3, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index_3, 'Price'] * bottleneck_1)
            
                    prices.append((min_price_index_3, current_df.loc[min_price_index_3, 'Price'], max_price_index_3, current_df.loc[max_price_index_3, 'Price'], profit, charge_level))
            
                  elif min_price_index_3 > max_price_index_3:
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
            
                        profit = (current_df.loc[max_price_index_3, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index_3, 'Price'] * bottleneck_1)
            
                        prices.append((min_price_index_3, current_df.loc[min_price_index_3, 'Price'], max_price_index_3, current_df.loc[max_price_index_3, 'Price'], profit, charge_level))
            
                  else:
                        continue         
                        
    
    

    return prices
run_electricity_strategy(df=Y_r, eff=1, capacity=5, ramp_rate=2, min_charge_level=1)

In [ ]:
def run_electricity_strategy(df, eff):
    prices = []
    level_0_values = df['level_0'].unique()

    for level_0 in level_0_values:
        current_df = df[df['level_0'] == level_0]
        max_price_index = current_df['Price'].idxmax()
        min_price_index = current_df['Price'].idxmin()

        smaller_index = min(min_price_index, max_price_index)
        larger_index = max(min_price_index, max_price_index)
        remaining_prices_0 = current_df[(current_df.index > smaller_index) & (current_df.index < larger_index)]
        remaining_prices_1 = current_df[(current_df.index > smaller_index) & (current_df.index > larger_index)]
        remaining_prices_2 = current_df[(current_df.index < smaller_index) & (current_df.index < larger_index)]

        if len(remaining_prices_0) > 1:
            max_price_index_1 = remaining_prices_0['Price'].idxmax()
            min_price_index_1 = remaining_prices_0['Price'].idxmin()
            if current_df.loc[min_price_index_1, 'Price'] < current_df.loc[max_price_index, 'Price'] * eff:
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'], 
                               max_price_index, current_df.loc[max_price_index, 'Price']))

                prices.append((min_price_index_1, current_df.loc[min_price_index_1, 'Price'], 
                               max_price_index_1, current_df.loc[max_price_index_1, 'Price']))

                smaller_index_1 = min(min_price_index_1, max_price_index_1)
                larger_index_1 = max(min_price_index_1, max_price_index_1)
                remaining_prices_01 = current_df[(current_df.index > smaller_index_1) & (current_df.index < larger_index_1)]
                if len(remaining_prices_01) > 1:
                    max_price_index_4 = remaining_prices_01['Price'].idxmax()
                    min_price_index_4 = remaining_prices_01['Price'].idxmin()
                    if current_df.loc[min_price_index_4, 'Price'] < current_df.loc[max_price_index_1, 'Price'] * eff:
                        prices.append((min_price_index_4, current_df.loc[min_price_index_4, 'Price'], 
                                       max_price_index_4, current_df.loc[max_price_index_4, 'Price']))           
        if len(remaining_prices_1) > 1:
            max_price_index_2 = remaining_prices_1['Price'].idxmax()
            min_price_index_2 = remaining_prices_1['Price'].idxmin()
            if current_df.loc[min_price_index_2, 'Price'] < current_df.loc[max_price_index_1, 'Price'] * eff:
                prices.append((min_price_index_2, current_df.loc[min_price_index_2, 'Price'], 
                               max_price_index_2, current_df.loc[max_price_index_2, 'Price']))

        if len(remaining_prices_2) > 1:
            max_price_index_3 = remaining_prices_2['Price'].idxmax()
            min_price_index_3 = remaining_prices_2['Price'].idxmin()
            if current_df.loc[min_price_index_3, 'Price'] < current_df.loc[max_price_index_3, 'Price'] * eff:
                prices.append((min_price_index_3, current_df.loc[min_price_index_3, 'Price'], 
                               max_price_index_3, current_df.loc[max_price_index_3, 'Price']))

                               
    return prices

run_electricity_strategy(df=Y_r, eff=1)